In [1]:
import os

# Set your own Mistral API key here
os.environ["MISTRAL_API_KEY"] = "NX1TKuFMtLERziU9xYXGM0cu202ZaXXX"

In [21]:
import os
import base64
import re
from mistralai import Mistral

def pdf_to_markdown(
    pdf_path: str,
    output_markdown_path: str,
    image_folder_name: str = "images",
    start_page: int = None,  # New parameter: starting page for conversion (inclusive)
    end_page: int = None     # New parameter: ending page for conversion (inclusive)
):
    """
    Converts a PDF file to Markdown using Mistral OCR, saving extracted images locally.
    Markdown image links will be updated to reference these local image files.

    Args:
        pdf_path (str): The path to the input PDF file.
        output_markdown_path (str): The path where the output Markdown file will be saved.
        image_folder_name (str): The name of the folder to save extracted images.
                                  This folder will be created relative to the script's execution directory.
        start_page (int, optional): The starting page number for OCR processing (1-indexed).
                                    If None, processing starts from the first page.
        end_page (int, optional): The ending page number for OCR processing (1-indexed).
                                  If None, processing goes to the last page.
    """
    try:
        # Retrieve the Mistral API key from environment variables
        api_key = os.environ.get("MISTRAL_API_KEY")
        if not api_key:
            print("Error: MISTRAL_API_KEY environment variable not set. Please set it before running.")
            return

        # Initialize the Mistral client
        client = Mistral(api_key=api_key)

        print(f"Starting PDF processing for: '{pdf_path}'...")
        if start_page is not None and end_page is not None:
            print(f"Processing pages from {start_page} to {end_page}...")
        elif start_page is not None:
            print(f"Processing from page {start_page} to the end...")
        elif end_page is not None:
            print(f"Processing from the beginning to page {end_page}...")


        # Open the PDF file in binary read mode
        with open(pdf_path, "rb") as f:
            # Prepare document parameters, including page range if specified
            print("start doc processing")
            document_params = {
                "type": "document_file",
                "document_file": f  # Pass the file object directly
            }
            if start_page is not None:
                document_params["start_page"] = start_page
            if end_page is not None:
                document_params["end_page"] = end_page

            # Call the Mistral OCR API to process the document
            # 'document_file' type is used for local file uploads
            # 'include_image_base64=True' ensures image data is returned
            ocr_response = client.ocr.process(
                model="mistral-ocr-latest",
                document=document_params, # Use the updated document_params
                include_image_base64=True
            )

            print("After API call")

        if not ocr_response:
            print("Error: No response received from Mistral OCR. The API call might have failed or returned empty.")
            return

        # Extract markdown content and image data from the OCR response
        markdown_content = ocr_response.markdown
        extracted_images = ocr_response.images

        # Create the directory for saving images if it doesn't exist
        if not os.path.exists(image_folder_name):
            os.makedirs(image_folder_name)
            print(f"Created image folder: '{image_folder_name}'")

        # Dictionary to map Mistral image IDs to their new local filenames
        image_id_to_filename = {}

        # Process and save extracted images
        if extracted_images:
            print(f"Found {len(extracted_images)} images. Saving them to '{image_folder_name}'...")
            for img_data in extracted_images:
                image_id = img_data.id
                image_base64 = img_data.image_base64

                if image_base64:
                    try:
                        # Construct a filename for the image (assuming PNG format)
                        image_filename = f"{image_id}.png"
                        image_full_path = os.path.join(image_folder_name, image_filename)
                        

                        # Decode the base64 image data and write it to a file
                        with open(image_full_path, "wb") as img_file:
                            img_file.write(base64.b64decode(image_base64))

                        # Store the mapping for later markdown modification
                        image_id_to_filename[image_id] = image_filename
                        print(f"Saved image: '{image_full_path}'")
                    except Exception as e:
                        print(f"Error saving image '{image_id}': {e}")
                else:
                    print(f"Warning: Image '{image_id}' has no base64 data and could not be saved.")
        else:
            print("No images were extracted from the PDF.")

        # Modify markdown content to reference local image paths
        if markdown_content:
            print("Adjusting image references in markdown content...")
            for image_id, filename in image_id_to_filename.items():
                # Regular expression to find Markdown image links like `![alt text](image_id)`
                # `re.escape(image_id)` is used to handle special characters in image_id if any.
                # `(.*?)` captures the alt text (non-greedy).
                # `\\1` in the replacement refers to the captured alt text.
                markdown_content = re.sub(
                    rf'\!\[(.*?)\]\({re.escape(image_id)}\)',
                    rf'![\\1]({image_folder_name}/{filename})',
                    markdown_content
                )
                # Also handle cases where there might be no alt text, e.g., `![]({image_id})`
                markdown_content = re.sub(
                    rf'\!\[\]\({re.escape(image_id)}\)',
                    rf'[]({image_folder_name}/{filename})',
                    markdown_content
                )

            # Save the modified markdown content to the specified output file
            with open(output_markdown_path, "w", encoding="utf-8") as md_file:
                md_file.write(markdown_content)
            print(f"Markdown content saved to: '{output_markdown_path}'")
        else:
            print("No markdown content was extracted from the PDF.")

    except FileNotFoundError:
        print(f"Error: The PDF file '{pdf_path}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        # For more detailed debugging, uncomment the following lines:
        # import traceback
        # traceback.print_exc()

# --- Example Usage ---
if __name__ == "__main__":
    # IMPORTANT:
    # 1. Install the Mistral AI library: pip install mistralai
    # 2. Set your Mistral API key as an environment variable named MISTRAL_API_KEY.
    #    - On Linux/macOS: export MISTRAL_API_KEY="your_api_key_here"
    #    - On Windows (Command Prompt): set MISTRAL_API_KEY="your_api_key_here"
    #    - On Windows (PowerShell): $env:MISTRAL_API_KEY="your_api_key_here"
    # 3. Replace 'path/to/your/document.pdf' with the actual path to your PDF file.
    #    You can download a sample PDF, for example, from an arXiv paper:
    #    https://arxiv.org/pdf/2201.04234.pdf (save it locally as 'sample.pdf')

    # Define your input and output paths
    input_pdf_file = "sample.pdf"  # Replace with your PDF file path
    output_md_file = "output.md"
    output_image_folder = "images" # Folder where images will be saved

    # Example 1: Convert the entire PDF
    print("--- Converting entire PDF ---")
    pdf_to_markdown(input_pdf_file, output_md_file, output_image_folder)

    # # Example 2: Convert a specific page range (e.g., pages 1 to 3)
    # print("\n--- Converting pages 1 to 3 ---")
    # pdf_to_markdown(input_pdf_file, "output_pages_1_3.md", output_image_folder, start_page=1, end_page=3)

    # # Example 3: Convert from a specific page to the end (e.g., from page 2 onwards)
    # print("\n--- Converting from page 2 to end ---")
    # pdf_to_markdown(input_pdf_file, "output_from_page_2.md", output_image_folder, start_page=2)

    # # Example 4: Convert up to a specific page (e.g., up to page 5)
    # print("\n--- Converting up to page 5 ---")
    # pdf_to_markdown(input_pdf_file, "output_up_to_page_5.md", output_image_folder, end_page=5)

    # print("\nProcess completed. Check the generated markdown files and image folder.")

--- Converting entire PDF ---
Starting PDF processing for: 'sample.pdf'...
start doc processing
An unexpected error occurred: 6 validation errors for Unmarshaller
body.FileChunk.file_id
  Field required [type=missing, input_value={'type': 'document_file',...ader name='sample.pdf'>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
body.FileChunk.type
  Input should be 'file' [type=literal_error, input_value='document_file', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/literal_error
body.ImageURLChunk.image_url
  Field required [type=missing, input_value={'type': 'document_file',...ader name='sample.pdf'>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
body.ImageURLChunk.type
  Input should be 'image_url' [type=literal_error, input_value='document_file', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/literal_error
body

In [25]:
import os
import base64
import re
from mistralai import Mistral

def pdf_to_markdown(
    pdf_path: str,
    output_markdown_path: str,
    image_folder_name: str = "images",
    start_page: int = None,  # New parameter: starting page for conversion (inclusive)
    end_page: int = None     # New parameter: ending page for conversion (inclusive)
):
    """
    Converts a PDF file to Markdown using Mistral OCR, saving extracted images locally.
    Markdown image links will be updated to reference these local image files.

    Args:
        pdf_path (str): The path to the input PDF file.
        output_markdown_path (str): The path where the output Markdown file will be saved.
        image_folder_name (str): The name of the folder to save extracted images.
                                  This folder will be created relative to the script's execution directory.
        start_page (int, optional): The starting page number for OCR processing (1-indexed).
                                    If None, processing starts from the first page.
        end_page (int, optional): The ending page number for OCR processing (1-indexed).
                                  If None, processing goes to the last page.
    """
    try:
        # Retrieve the Mistral API key from environment variables
        api_key = os.environ.get("MISTRAL_API_KEY")
        if not api_key:
            print("Error: MISTRAL_API_KEY environment variable not set. Please set it before running.")
            return

        # Initialize the Mistral client
        client = Mistral(api_key=api_key)

        print(f"Starting PDF processing for: '{pdf_path}'...")
        if start_page is not None and end_page is not None:
            print(f"Processing pages from {start_page} to {end_page}...")
        elif start_page is not None:
            print(f"Processing from page {start_page} to the end...")
        elif end_page is not None:
            print(f"Processing from the beginning to page {end_page}...")


        # Open the PDF file in binary read mode
        with open(pdf_path, "rb") as f:
            # Call the Mistral OCR API to process the document
            # When uploading a local file, use 'document_file' keyword argument directly.
            # 'start_page' and 'end_page' are also passed as direct keyword arguments.
            ocr_response = client.ocr.process(
                model="mistral-ocr-latest",
                document_file=f, # Pass the file object directly here
                start_page=start_page, # Pass start_page directly
                end_page=end_page,     # Pass end_page directly
                include_image_base64=True
            )

        if not ocr_response:
            print("Error: No response received from Mistral OCR. The API call might have failed or returned empty.")
            return

        # Extract markdown content and image data from the OCR response
        markdown_content = ocr_response.markdown
        extracted_images = ocr_response.images

        # Create the directory for saving images if it doesn't exist
        if not os.path.exists(image_folder_name):
            os.makedirs(image_folder_name)
            print(f"Created image folder: '{image_folder_name}'")

        # Dictionary to map Mistral image IDs to their new local filenames
        image_id_to_filename = {}

        # Process and save extracted images
        if extracted_images:
            print(f"Found {len(extracted_images)} images. Saving them to '{image_folder_name}'...")
            for img_data in extracted_images:
                image_id = img_data.id
                image_base64 = img_data.image_base64

                if image_base64:
                    try:
                        # Construct a filename for the image (assuming PNG format)
                        image_filename = f"{image_id}.png"
                        image_full_path = os.path.join(image_folder_name, image_filename)

                        # Decode the base64 image data and write it to a file
                        with open(image_full_path, "wb") as img_file:
                            img_file.write(base64.b64decode(image_base64))

                        # Store the mapping for later markdown modification
                        image_id_to_filename[image_id] = image_filename
                        print(f"Saved image: '{image_full_path}'")
                    except Exception as e:
                        print(f"Error saving image '{image_id}': {e}")
                else:
                    print(f"Warning: Image '{image_id}' has no base64 data and could not be saved.")
        else:
            print("No images were extracted from the PDF.")

        # Modify markdown content to reference local image paths
        if markdown_content:
            print("Adjusting image references in markdown content...")
            for image_id, filename in image_id_to_filename.items():
                # Regular expression to find Markdown image links like `![alt text](image_id)`
                # `re.escape(image_id)` is used to handle special characters in image_id if any.
                # `(.*?)` captures the alt text (non-greedy).
                # `\\1` in the replacement refers to the captured alt text.
                markdown_content = re.sub(
                    rf'\!\[(.*?)\]\({re.escape(image_id)}\)',
                    rf'![\\1]({image_folder_name}/{filename})',
                    markdown_content
                )
                # Also handle cases where there might be no alt text, e.g., `![]({image_id})`
                markdown_content = re.sub(
                    rf'\!\[\]\({re.escape(image_id)}\)',
                    rf'[]({image_folder_name}/{filename})',
                    markdown_content
                )

            # Save the modified markdown content to the specified output file
            with open(output_markdown_path, "w", encoding="utf-8") as md_file:
                md_file.write(markdown_content)
            print(f"Markdown content saved to: '{output_markdown_path}'")
        else:
            print("No markdown content was extracted from the PDF.")

    except FileNotFoundError:
        print(f"Error: The PDF file '{pdf_path}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        # For more detailed debugging, uncomment the following lines:
        # import traceback
        # traceback.print_exc()

# --- Example Usage ---
if __name__ == "__main__":
    # IMPORTANT:
    # 1. Install the Mistral AI library: pip install mistralai
    # 2. Set your Mistral API key as an environment variable named MISTRAL_API_KEY.
    #    - On Linux/macOS: export MISTRAL_API_KEY="your_api_key_here"
    #    - On Windows (Command Prompt): set MISTRAL_API_KEY="your_api_key_here"
    #    - On Windows (PowerShell): $env:MISTRAL_API_KEY="your_api_key_here"
    # 3. Replace 'path/to/your/document.pdf' with the actual path to your PDF file.
    #    You can download a sample PDF, for example, from an arXiv paper:
    #    https://arxiv.org/pdf/2201.04234.pdf (save it locally as 'sample.pdf')

    # Define your input and output paths
    input_pdf_file = "sample.pdf"  # Replace with your PDF file path
    output_md_file = "output.md"
    output_image_folder = "extracted_images" # Folder where images will be saved

    # Example 1: Convert the entire PDF
    print("--- Converting entire PDF ---")
    pdf_to_markdown(input_pdf_file, output_md_file, output_image_folder)

    # # Example 2: Convert a specific page range (e.g., pages 1 to 3)
    # print("\n--- Converting pages 1 to 3 ---")
    # pdf_to_markdown(input_pdf_file, "output_pages_1_3.md", output_image_folder, start_page=1, end_page=3)

    # # Example 3: Convert from a specific page to the end (e.g., from page 2 onwards)
    # print("\n--- Converting from page 2 to end ---")
    # pdf_to_markdown(input_pdf_file, "output_from_page_2.md", output_image_folder, start_page=2)

    # # Example 4: Convert up to a specific page (e.g., up to page 5)
    # print("\n--- Converting up to page 5 ---")
    # pdf_to_markdown(input_pdf_file, "output_up_to_page_5.md", output_image_folder, end_page=5)

    # print("\nProcess completed. Check the generated markdown files and image folder.")

--- Converting entire PDF ---
Starting PDF processing for: 'sample.pdf'...
An unexpected error occurred: Ocr.process() got an unexpected keyword argument 'document_file'


In [27]:
import os
import base64
import re
from mistralai import Mistral

def pdf_to_markdown(
    pdf_path: str,
    output_markdown_path: str,
    image_folder_name: str = "images",
    start_page: int = None,  # New parameter: starting page for conversion (inclusive)
    end_page: int = None     # New parameter: ending page for conversion (inclusive)
):
    """
    Converts a PDF file to Markdown using Mistral OCR, saving extracted images locally.
    Markdown image links will be updated to reference these local image files.

    Args:
        pdf_path (str): The path to the input PDF file.
        output_markdown_path (str): The path where the output Markdown file will be saved.
        image_folder_name (str): The name of the folder to save extracted images.
                                  This folder will be created relative to the script's execution directory.
        start_page (int, optional): The starting page number for OCR processing (1-indexed).
                                    If None, processing starts from the first page.
        end_page (int, optional): The ending page number for OCR processing (1-indexed).
                                  If None, processing goes to the last page.
    """
    try:
        # Retrieve the Mistral API key from environment variables
        api_key = os.environ.get("MISTRAL_API_KEY")
        if not api_key:
            print("Error: MISTRAL_API_KEY environment variable not set. Please set it before running.")
            return

        # Initialize the Mistral client
        client = Mistral(api_key=api_key)

        print(f"Starting PDF processing for: '{pdf_path}'...")
        if start_page is not None and end_page is not None:
            print(f"Processing pages from {start_page} to {end_page}...")
        elif start_page is not None:
            print(f"Processing from page {start_page} to the end...")
        elif end_page is not None:
            print(f"Processing from the beginning to page {end_page}...")


        # Open the PDF file in binary read mode
        with open(pdf_path, "rb") as f:
            # Prepare the document dictionary, including page range if specified
            document_payload = {
                "type": "file",  # Correct type for local file upload
                "data": f        # Pass the file object here
            }
            if start_page is not None:
                document_payload["start_page"] = start_page
            if end_page is not None:
                document_payload["end_page"] = end_page

            # Call the Mistral OCR API to process the document
            # The 'document' parameter should be a dictionary containing file data and page range.
            ocr_response = client.ocr.process(
                model="mistral-ocr-latest",
                document=document_payload, # Pass the constructed dictionary here
                include_image_base64=True
            )

        if not ocr_response:
            print("Error: No response received from Mistral OCR. The API call might have failed or returned empty.")
            return

        # Extract markdown content and image data from the OCR response
        markdown_content = ocr_response.markdown
        extracted_images = ocr_response.images

        # Create the directory for saving images if it doesn't exist
        if not os.path.exists(image_folder_name):
            os.makedirs(image_folder_name)
            print(f"Created image folder: '{image_folder_name}'")

        # Dictionary to map Mistral image IDs to their new local filenames
        image_id_to_filename = {}

        # Process and save extracted images
        if extracted_images:
            print(f"Found {len(extracted_images)} images. Saving them to '{image_folder_name}'...")
            for img_data in extracted_images:
                image_id = img_data.id
                image_base64 = img_data.image_base64

                if image_base64:
                    try:
                        # Construct a filename for the image (assuming PNG format)
                        image_filename = f"{image_id}.png"
                        image_full_path = os.path.join(image_folder_name, image_filename)

                        # Decode the base64 image data and write it to a file
                        with open(image_full_path, "wb") as img_file:
                            img_file.write(base64.b64decode(image_base64))

                        # Store the mapping for later markdown modification
                        image_id_to_filename[image_id] = image_filename
                        print(f"Saved image: '{image_full_path}'")
                    except Exception as e:
                        print(f"Error saving image '{image_id}': {e}")
                else:
                    print(f"Warning: Image '{image_id}' has no base64 data and could not be saved.")
        else:
            print("No images were extracted from the PDF.")

        # Modify markdown content to reference local image paths
        if markdown_content:
            print("Adjusting image references in markdown content...")
            for image_id, filename in image_id_to_filename.items():
                # Regular expression to find Markdown image links like `![alt text](image_id)`
                # `re.escape(image_id)` is used to handle special characters in image_id if any.
                # `(.*?)` captures the alt text (non-greedy).
                # `\\1` in the replacement refers to the captured alt text.
                markdown_content = re.sub(
                    rf'\!\[(.*?)\]\({re.escape(image_id)}\)',
                    rf'![\\1]({image_folder_name}/{filename})',
                    markdown_content
                )
                # Also handle cases where there might be no alt text, e.g., `![]({image_id})`
                markdown_content = re.sub(
                    rf'\!\[\]\({re.escape(image_id)}\)',
                    rf'[]({image_folder_name}/{filename})',
                    markdown_content
                )

            # Save the modified markdown content to the specified output file
            with open(output_markdown_path, "w", encoding="utf-8") as md_file:
                md_file.write(markdown_content)
            print(f"Markdown content saved to: '{output_markdown_path}'")
        else:
            print("No markdown content was extracted from the PDF.")

    except FileNotFoundError:
        print(f"Error: The PDF file '{pdf_path}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        # For more detailed debugging, uncomment the following lines:
        # import traceback
        # traceback.print_exc()

# --- Example Usage ---
if __name__ == "__main__":
    # IMPORTANT:
    # 1. Install the Mistral AI library: pip install mistralai
    # 2. Set your Mistral API key as an environment variable named MISTRAL_API_KEY.
    #    - On Linux/macOS: export MISTRAL_API_KEY="your_api_key_here"
    #    - On Windows (Command Prompt): set MISTRAL_API_KEY="your_api_key_here"
    #    - On PowerShell: $env:MISTRAL_API_KEY="your_api_key_here"
    # 3. Replace 'path/to/your/document.pdf' with the actual path to your PDF file.
    #    You can download a sample PDF, for example, from an arXiv paper:
    #    https://arxiv.org/pdf/2201.04234.pdf (save it locally as 'sample.pdf')

    # Define your input and output paths
    input_pdf_file = "sample.pdf"  # Replace with your PDF file path
    output_md_file = "output.md"
    output_image_folder = "extracted_images" # Folder where images will be saved

    # Example 1: Convert the entire PDF
    print("--- Converting entire PDF ---")
    pdf_to_markdown(input_pdf_file, output_md_file, output_image_folder)

--- Converting entire PDF ---
Starting PDF processing for: 'sample.pdf'...
An unexpected error occurred: 5 validation errors for Unmarshaller
body.FileChunk.file_id
  Field required [type=missing, input_value={'type': 'file', 'data': ...ader name='sample.pdf'>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
body.ImageURLChunk.image_url
  Field required [type=missing, input_value={'type': 'file', 'data': ...ader name='sample.pdf'>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
body.ImageURLChunk.type
  Input should be 'image_url' [type=literal_error, input_value='file', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/literal_error
body.DocumentURLChunk.document_url
  Field required [type=missing, input_value={'type': 'file', 'data': ...ader name='sample.pdf'>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
b

In [32]:
import os
import base64
from pathlib import Path
from mistralai import Mistral
import re

class PDFToMarkdownConverter:
    def __init__(self, api_key=None):
        """
        Initialize the PDF to Markdown converter with Mistral OCR
        
        Args:
            api_key (str): Mistral API key. If None, will use MISTRAL_API_KEY environment variable
        """
        self.api_key = api_key or os.environ.get("MISTRAL_API_KEY")
        if not self.api_key:
            raise ValueError("API key is required. Set MISTRAL_API_KEY environment variable or pass api_key parameter.")
        
        self.client = Mistral(api_key=self.api_key)
    
    def convert_pdf_to_markdown(self, pdf_path, output_dir=None):
        """
        Convert a PDF file to markdown with images saved in a separate folder
        
        Args:
            pdf_path (str): Path to the PDF file
            output_dir (str): Directory to save output files. If None, uses PDF file directory
            
        Returns:
            tuple: (markdown_content, markdown_file_path, images_saved)
        """
        pdf_path = Path(pdf_path)
        
        if not pdf_path.exists():
            raise FileNotFoundError(f"PDF file not found: {pdf_path}")
        
        # Set up output directory
        if output_dir is None:
            output_dir = pdf_path.parent
        else:
            output_dir = Path(output_dir)
        
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Create images folder
        images_dir = output_dir / "images"
        images_dir.mkdir(exist_ok=True)
        
        # Base name for output files
        base_name = pdf_path.stem
        
        print(f"Processing PDF: {pdf_path}")
        print(f"Output directory: {output_dir}")
        print(f"Images directory: {images_dir}")
        
        try:
            # First, upload the PDF file
            print("Uploading PDF file...")
            with open(pdf_path, 'rb') as file:
                upload_response = self.client.files.upload(
                    file=file,
                    purpose="fine-tune"  # or another valid purpose
                )
            
            file_id = upload_response.id
            print(f"File uploaded with ID: {file_id}")
            
            # Process PDF with Mistral OCR using the uploaded file
            print("Processing PDF with Mistral OCR...")
            ocr_response = self.client.ocr.process(
                model="mistral-ocr-latest",
                document={
                    "type": "file",
                    "file_id": file_id
                },
                include_image_base64=True
            )
            
            # Extract markdown content
            markdown_content = ocr_response.markdown
            
            # Process and save images
            images_saved = 0
            if hasattr(ocr_response, 'images') and ocr_response.images:
                images_saved = self._save_images(ocr_response.images, images_dir, base_name)
                
                # Update markdown to reference local images
                markdown_content = self._update_image_references(markdown_content, images_dir.name)
            
            # Save markdown file
            markdown_file_path = output_dir / f"{base_name}.md"
            with open(markdown_file_path, 'w', encoding='utf-8') as f:
                f.write(markdown_content)
            
            print(f"✅ Conversion completed!")
            print(f"📄 Markdown saved: {markdown_file_path}")
            print(f"🖼️  Images saved: {images_saved}")
            
            # Clean up uploaded file
            try:
                self.client.files.delete(file_id)
                print(f"🧹 Cleaned up uploaded file: {file_id}")
            except Exception as cleanup_error:
                print(f"⚠️  Warning: Could not clean up uploaded file: {cleanup_error}")
            
            return markdown_content, markdown_file_path, images_saved
            
        except Exception as e:
            print(f"❌ Error during conversion: {str(e)}")
            raise
    
    def _save_images(self, images, images_dir, base_name):
        """
        Save base64 encoded images to the images directory
        
        Args:
            images (list): List of image data from OCR response
            images_dir (Path): Directory to save images
            base_name (str): Base name for image files
            
        Returns:
            int: Number of images saved
        """
        images_saved = 0
        
        for i, image_data in enumerate(images):
            try:
                # Handle different possible image data structures
                if isinstance(image_data, dict):
                    if 'base64' in image_data:
                        base64_data = image_data['base64']
                        # Get format from metadata or default to png
                        image_format = image_data.get('format', 'png')
                    elif 'data' in image_data:
                        base64_data = image_data['data']
                        image_format = image_data.get('format', 'png')
                    else:
                        base64_data = str(image_data)
                        image_format = 'png'
                else:
                    base64_data = str(image_data)
                    image_format = 'png'
                
                # Clean base64 data (remove data URL prefix if present)
                if base64_data.startswith('data:'):
                    base64_data = base64_data.split(',', 1)[1]
                
                # Generate filename
                image_filename = f"{base_name}_image_{i+1:03d}.{image_format}"
                image_path = images_dir / image_filename
                
                # Decode and save image
                image_bytes = base64.b64decode(base64_data)
                with open(image_path, 'wb') as f:
                    f.write(image_bytes)
                
                images_saved += 1
                print(f"💾 Saved image: {image_filename}")
                
            except Exception as e:
                print(f"⚠️  Warning: Could not save image {i+1}: {str(e)}")
                continue
        
        return images_saved
    
    def _update_image_references(self, markdown_content, images_folder_name):
        """
        Update image references in markdown to point to local images folder
        
        Args:
            markdown_content (str): Original markdown content
            images_folder_name (str): Name of the images folder
            
        Returns:
            str: Updated markdown content
        """
        # Pattern to match various image reference formats
        patterns = [
            r'!\[([^\]]*)\]\(([^)]+)\)',  # ![alt](url)
            r'<img[^>]+src=["\']([^"\']+)["\'][^>]*>',  # HTML img tags
        ]
        
        updated_content = markdown_content
        
        for pattern in patterns:
            def replace_image_ref(match):
                if pattern.startswith('!'):
                    # Markdown format
                    alt_text = match.group(1)
                    original_url = match.group(2)
                    
                    # Generate local image reference
                    # This is a simplified approach - in practice, you'd need to map
                    # the original references to the saved images
                    local_ref = f"{images_folder_name}/image_{hash(original_url) % 1000:03d}.png"
                    return f"![{alt_text}]({local_ref})"
                else:
                    # HTML format - keep as is for now
                    return match.group(0)
            
            updated_content = re.sub(pattern, replace_image_ref, updated_content)
        
        return updated_content

def main():
    """
    Example usage of the PDF to Markdown converter
    """
    # Initialize converter
    converter = PDFToMarkdownConverter()
    
    # Example usage
    pdf_file = input("Enter path to PDF file: ").strip()
    
    if not pdf_file:
        print("No file specified. Using example...")
        return
    
    try:
        # Convert PDF to markdown
        markdown_content, markdown_file, images_count = converter.convert_pdf_to_markdown(pdf_file)
        
        print(f"\n🎉 Success! Converted '{pdf_file}' to markdown")
        print(f"📁 Output: {markdown_file}")
        print(f"🖼️  Images extracted: {images_count}")
        
        # Optionally display first few lines of markdown
        print(f"\n📝 First 300 characters of markdown:")
        print("-" * 50)
        print(markdown_content[:300] + "..." if len(markdown_content) > 300 else markdown_content)
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")

if __name__ == "__main__":
    main()

Enter path to PDF file:  sample.pdf


Processing PDF: sample.pdf
Output directory: .
Images directory: images
Uploading PDF file...
❌ Error during conversion: 1 validation error for Unmarshaller
body
  Input should be a valid dictionary or instance of File [type=model_type, input_value=<_io.BufferedReader name='sample.pdf'>, input_type=BufferedReader]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
❌ Error: 1 validation error for Unmarshaller
body
  Input should be a valid dictionary or instance of File [type=model_type, input_value=<_io.BufferedReader name='sample.pdf'>, input_type=BufferedReader]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type


In [36]:
import os
import base64
from mistralai import Mistral
from dotenv import load_dotenv

def save_images(images, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    saved_files = []
    for img in images:
        img_filename = img.id  # unique name like "img-0.jpeg"
        filepath = os.path.join(output_dir, img_filename)
        with open(filepath, "wb") as f:
            f.write(base64.b64decode(img.image_base64))
        saved_files.append(img_filename)
    return saved_files

def pdf_to_markdown(pdf_path, md_output="sample.md", img_dir="images"):
    load_dotenv()
    api_key = os.environ["MISTRAL_API_KEY"]
    client = Mistral(api_key=api_key)

    # Encode PDF to base64 data URI
    with open(pdf_path, "rb") as f:
        pdf_b64 = base64.b64encode(f.read()).decode("utf-8")
    data_uri = f"data:application/pdf;base64,{pdf_b64}"

    resp = client.ocr.process(
        model="mistral-ocr-latest",
        document={"type": "document_url", "document_url": data_uri},
        include_image_base64=True
    )

    total_md = ""
    with open(md_output, "w", encoding="utf-8") as md:
        for page in resp.pages:
            saved = save_images(page.images or [], img_dir)

            # rewrite image links to include folder prefix
            md_text = page.markdown
            for fname in saved:
                md_text = md_text.replace(f"({fname})", f"({img_dir}/{fname})")

            md.write(md_text + "\n\n")
            total_md += "\n\n"+md_text

    print(f"✅ Output saved to '{md_output}', with images in '{img_dir}/'")

    return total_md


In [37]:
import time

time1 = time.time()
total_md = pdf_to_markdown("sample.pdf")

print("Total time = ", time.time()-time1)

print(total_md)

✅ Output saved to 'sample.md', with images in 'images/'
Total time =  18.76445508003235


# Leveraging Unlabeled Data to Predict Out-of-Distribution Performance 

Saurabh Garg*<br>Carnegie Mellon University<br>sgarg2@andrew.cmu.edu Sivaraman Balakrishnan<br>Carnegie Mellon University<br>sbalakri@andrew.cmu.edu Zachary C. Lipton<br>Carnegie Mellon University<br>zlipton@andrew.cmu.edu

## Behnam Neyshabur

Google Research, Blueshift team neyshabur@google.com

## Hanie Sedghi

Google Research, Brain team hsedghi@google.com

## ABSTRACT

Real-world machine learning deployments are characterized by mismatches between the source (training) and target (test) distributions that may cause performance drops. In this work, we investigate methods for predicting the target domain accuracy using only labeled source data and unlabeled target data. We propose Average Thresholded Confidence (ATC), a practical method that learns a threshold on the model's confidence, predicting accuracy as the fraction 

In [2]:
import os

chapter_list = [
  {
    "chapter_or_appendix": "Preface",
    "title": "",
    "starting_page": 1
  },
  {
    "chapter_or_appendix": "Chapter 1",
    "title": "Essential Ideas",
    "starting_page": 9
  },
  {
    "chapter_or_appendix": "Chapter 2",
    "title": "Atoms, Molecules, and Ions",
    "starting_page": 67
  },
  {
    "chapter_or_appendix": "Chapter 3",
    "title": "Composition of Substances and Solutions",
    "starting_page": 131
  },
  {
    "chapter_or_appendix": "Chapter 4",
    "title": "Stoichiometry of Chemical Reactions",
    "starting_page": 175
  },
  {
    "chapter_or_appendix": "Chapter 5",
    "title": "Thermochemistry",
    "starting_page": 231
  },
  {
    "chapter_or_appendix": "Chapter 6",
    "title": "Electronic Structure and Periodic Properties of Elements",
    "starting_page": 281
  },
  {
    "chapter_or_appendix": "Chapter 7",
    "title": "Chemical Bonding and Molecular Geometry",
    "starting_page": 343
  },
  {
    "chapter_or_appendix": "Chapter 8",
    "title": "Advanced Theories of Covalent Bonding",
    "starting_page": 411
  },
  {
    "chapter_or_appendix": "Chapter 9",
    "title": "Gases",
    "starting_page": 457
  },
  {
    "chapter_or_appendix": "Chapter 10",
    "title": "Liquids and Solids",
    "starting_page": 521
  },
  {
    "chapter_or_appendix": "Chapter 11",
    "title": "Solutions and Colloids",
    "starting_page": 599
  },
  {
    "chapter_or_appendix": "Chapter 12",
    "title": "Kinetics",
    "starting_page": 657
  },
  {
    "chapter_or_appendix": "Chapter 13",
    "title": "Fundamental Equilibrium Concepts",
    "starting_page": 721
  },
  {
    "chapter_or_appendix": "Chapter 14",
    "title": "Acid-Base Equilibria",
    "starting_page": 763
  },
  {
    "chapter_or_appendix": "Chapter 15",
    "title": "Equilibria of Other Reaction Classes",
    "starting_page": 823
  },
  {
    "chapter_or_appendix": "Chapter 16",
    "title": "Thermodynamics",
    "starting_page": 861
  },
  {
    "chapter_or_appendix": "Chapter 17",
    "title": "Electrochemistry",
    "starting_page": 897
  },
  {
    "chapter_or_appendix": "Chapter 18",
    "title": "Representative Metals, Metalloids, and Nonmetals",
    "starting_page": 941
  },
  {
    "chapter_or_appendix": "Chapter 19",
    "title": "Transition Metals and Coordination Chemistry",
    "starting_page": 1029
  },
  {
    "chapter_or_appendix": "Chapter 20",
    "title": "Organic Chemistry",
    "starting_page": 1077
  },
  {
    "chapter_or_appendix": "Chapter 21",
    "title": "Nuclear Chemistry",
    "starting_page": 1127
  },
  {
    "chapter_or_appendix": "Appendix A",
    "title": "The Periodic Table",
    "starting_page": 1189
  },
  {
    "chapter_or_appendix": "Appendix B",
    "title": "Essential Mathematics",
    "starting_page": 1191
  },
  {
    "chapter_or_appendix": "Appendix C",
    "title": "Units and Conversion Factors",
    "starting_page": 1199
  },
  {
    "chapter_or_appendix": "Appendix D",
    "title": "Fundamental Physical Constants",
    "starting_page": 1201
  },
  {
    "chapter_or_appendix": "Appendix E",
    "title": "Water Properties",
    "starting_page": 1203
  },
  {
    "chapter_or_appendix": "Appendix F",
    "title": "Composition of Commercial Acids and Bases",
    "starting_page": 1209
  },
  {
    "chapter_or_appendix": "Appendix G",
    "title": "Standard Thermodynamic Properties for Selected Substances",
    "starting_page": 1211
  },
  {
    "chapter_or_appendix": "Appendix H",
    "title": "Ionization Constants of Weak Acids",
    "starting_page": 1225
  },
  {
    "chapter_or_appendix": "Appendix I",
    "title": "Ionization Constants of Weak Bases",
    "starting_page": 1229
  },
  {
    "chapter_or_appendix": "Appendix J",
    "title": "Solubility Products",
    "starting_page": 1231
  },
  {
    "chapter_or_appendix": "Appendix K",
    "title": "Formation Constants for Complex Ions",
    "starting_page": 1235
  },
  {
    "chapter_or_appendix": "Appendix L",
    "title": "Standard Electrode (Half-Cell) Potentials",
    "starting_page": 1237
  },
  {
    "chapter_or_appendix": "Appendix M",
    "title": "Half-Lives for Several Radioactive Isotopes",
    "starting_page": 1243
  },
  {
    "chapter_or_appendix": "Index",
    "title": "",
    "starting_page": 1313
  }
]

starting_pages = []
for chapter in chapter_list:
    chapter_file = chapter['chapter_or_appendix'].replace(" ","_")
    starting_pages.append((chapter_file, chapter['starting_page'] + 10))
    #print(starting_pages[-1])

In [3]:
import os
import base64
import re
import tempfile
import time
import shutil
from mistralai import Mistral
from dotenv import load_dotenv
from PyPDF2 import PdfReader, PdfWriter

def save_images(images, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    saved = []
    for img in images:
        b64 = img.image_base64
        if b64.startswith("data:image/"):
            b64 = b64.split(",", 1)[1]
        try:
            data = base64.b64decode(b64)
        except base64.binascii.Error as e:
            print(f"⚠️ Failed to decode image {img.id}: {e}")
            continue
        ext = os.path.splitext(img.id)[1].lstrip(".") or "jpg"
        fname = img.id if img.id.endswith(f".{ext}") else f"{img.id}.{ext}"
        with open(os.path.join(output_dir, fname), "wb") as f:
            f.write(data)
        saved.append(fname)
    return saved

def extract_pdf_pages(source_path, start_page, end_page):
    """
    Extracts specified page range from source_path and returns path to a temporary file.
    start_page and end_page are 1-based, inclusive.
    """
    reader = PdfReader(source_path)
    writer = PdfWriter()

    total_pages = len(reader.pages)
    start = max(1, start_page)
    end = min(end_page, total_pages)

    for i in range(start - 1, end):
        writer.add_page(reader.pages[i])

    temp_fd, temp_path = tempfile.mkstemp(suffix=".pdf")
    with os.fdopen(temp_fd, 'wb') as f:
        writer.write(f)

    return temp_path

def chunk_pdf(pdf_path, dest_pdf="Chapter.pdf",start_page=1, end_page=None):
    # Step 1: extract subset of PDF pages into a temporary file
    source_pdf = extract_pdf_pages(pdf_path, start_page, end_page or start_page)
    print(source_pdf)
    shutil.move(source_pdf, dest_pdf)

def pdf_to_markdown(pdf_path, md_output="output.md", img_dir="images", dest_pdf="test.pdf", start_page=1, end_page=None):
    load_dotenv()
    client = Mistral(api_key=os.environ["MISTRAL_API_KEY"])

    # Step 1: extract subset of PDF pages into a temporary file
    temp_pdf = extract_pdf_pages(pdf_path, start_page, end_page or start_page)

    # Step 2: Upload only the extracted portion
    with open(temp_pdf, "rb") as f:
        upload = client.files.upload(
            file={"file_name": "subset.pdf", "content": f},
            purpose="ocr"
        )

    # Step 3: Get signed URL
    signed = client.files.get_signed_url(file_id=upload.id)

    # Step 4: Process OCR (no need for page range since we already split)
    resp = client.ocr.process(
        model="mistral-ocr-latest",
        document={"type": "document_url", "document_url": signed.url},
        include_image_base64=True
    )

    # Step 5: Save Markdown and images
    doc_md = ""
    with open(md_output, "w", encoding="utf-8") as mdf:
        for i, page in enumerate(resp.pages, start=start_page):
            print(f"Processing page {i}")
            imgs = save_images(page.images or [], img_dir)
            markdown = page.markdown
            for f in imgs:
                markdown = re.sub(rf"!\[.*?\]\({re.escape(f)}\)", f"![Image]({img_dir}/{f})", markdown)
            mdf.write(markdown + "\n\n")
            doc_md += markdown + "\n\n"

    # Save temp file 
    shutil.move(temp_pdf, dest_pdf)

    print(f"✅ Markdown saved to '{md_output}' for pages {start_page}-{end_page}, images in '{img_dir}/'")
    return doc_md

In [5]:
for idx in range(len(starting_pages)-1):
    
    start_page = starting_pages[idx][1]
    dest_pdf = starting_pages[idx][0] + ".pdf"
    image_folder = starting_pages[idx][0] + "_images"
    md_file = starting_pages[idx][0] + ".md"
    if (idx+1 < len(starting_pages)):
        end_page = starting_pages[idx+1][1]-1
    else:
        end_page = 1331
    
    pdf_path = "Chemistry2e-OP.pdf"
    if os.path.exists(md_file):
        print(f"Chapter markdown {md_file} has been created. Skip it")
        continue

    time1 = time.time()
    print("dest_pdf, md_file, image_folder, start, end = ", dest_pdf, image_folder, start_page, end_page)
    #chunk_pdf(pdf_path, dest_pdf, start_page, end_page)
    pdf_to_markdown(pdf_path, md_file, image_folder, dest_pdf, start_page, end_page)
    
    print("Time = ", time.time()-time1)

Chapter markdown Preface.md has been created. Skip it
Chapter markdown Chapter_1.md has been created. Skip it
Chapter markdown Chapter_2.md has been created. Skip it
Chapter markdown Chapter_3.md has been created. Skip it
dest_pdf, md_file, image_folder, start, end =  Chapter_4.pdf Chapter_4_images 185 240
Processing page 185
Processing page 186
Processing page 187
Processing page 188
Processing page 189
Processing page 190
Processing page 191
Processing page 192
Processing page 193
Processing page 194
Processing page 195
Processing page 196
Processing page 197
Processing page 198
Processing page 199
Processing page 200
Processing page 201
Processing page 202
Processing page 203
Processing page 204
Processing page 205
Processing page 206
Processing page 207
Processing page 208
Processing page 209
Processing page 210
Processing page 211
Processing page 212
Processing page 213
Processing page 214
Processing page 215
Processing page 216
Processing page 217
Processing page 218
Processing p

# Chapter 1 

## Essential Ideas

![Image](Chapter_1_images/img-0.jpeg)

Figure 1.1 Chemical substances and processes are essential for our existence, providing sustenance, keeping us clean and healthy, fabricating electronic devices, enabling transportation, and much more. (credit "left": modification of work by "vxla"/Flickr; credit "left middle": modification of work by "the Italian voice"/Flickr; credit "right middle": modification of work by Jason Trim; credit "right": modification of work by "gosheshe"/Flickr)

## Chapter Outline

1.1 Chemistry in Context
1.2 Phases and Classification of Matter
1.3 Physical and Chemical Properties
1.4 Measurements
1.5 Measurement Uncertainty, Accuracy, and Precision
1.6 Mathematical Treatment of Measurement Results

## Introduction

Your alarm goes off and, after hitting "snooze" once or twice, you pry yourself out of bed. You make a cup of coffee to help you get going, and then you shower, get dressed, eat breakfast, and check your phone for messages. On your way to school, you stop to fill your car's gas tank, almost making you late for the first day of chemistry class. As you find a seat in the classroom, you read the question projected on the screen: "Welcome to class! Why should we study chemistry?"

Do you have an answer? You may be studying chemistry because it fulfills an academic requirement, but if you consider your daily activities, you might find chemistry interesting for other reasons. Most everything you do and encounter during your day involves chemistry. Making coffee, cooking eggs, and toasting bread involve chemistry. The products you use-like soap and shampoo, the fabrics you wear, the electronics that keep you connected to your world, the gasoline that propels your car-all of these and more involve chemical substances and processes. Whether you are aware or not, chemistry is part of your everyday world. In this course, you will learn many of the essential principles underlying the chemistry of modern-day life.

# 1.1 Chemistry in Context 

By the end of this module, you will be able to:

- Outline the historical development of chemistry
- Provide examples of the importance of chemistry in everyday life
- Describe the scientific method
- Differentiate among hypotheses, theories, and laws
- Provide examples illustrating macroscopic, microscopic, and symbolic domains

Throughout human history, people have tried to convert matter into more useful forms. Our Stone Age ancestors chipped pieces of flint into useful tools and carved wood into statues and toys. These endeavors involved changing the shape of a substance without changing the substance itself. But as our knowledge increased, humans began to change the composition of the substances as well-clay was converted into pottery, hides were cured to make garments, copper ores were transformed into copper tools and weapons, and grain was made into bread.
Humans began to practice chemistry when they learned to control fire and use it to cook, make pottery, and smelt metals. Subsequently, they began to separate and use specific components of matter. A variety of drugs such as aloe, myrrh, and opium were isolated from plants. Dyes, such as indigo and Tyrian purple, were extracted from plant and animal matter. Metals were combined to form alloys-for example, copper and tin were mixed together to make bronze-and more elaborate smelting techniques produced iron. Alkalis were extracted from ashes, and soaps were prepared by combining these alkalis with fats. Alcohol was produced by fermentation and purified by distillation.
Attempts to understand the behavior of matter extend back for more than 2500 years. As early as the sixth century BC, Greek philosophers discussed a system in which water was the basis of all things. You may have heard of the Greek postulate that matter consists of four elements: earth, air, fire, and water. Subsequently, an amalgamation of chemical technologies and philosophical speculations was spread from Egypt, China, and the eastern Mediterranean by alchemists, who endeavored to transform "base metals" such as lead into "noble metals" like gold, and to create elixirs to cure disease and extend life (Figure 1.2).

![Image](Chapter_1_images/img-1.jpeg)

Figure 1.2 This portrayal shows an alchemist's workshop circa 1580. Although alchemy made some useful contributions to how to manipulate matter, it was not scientific by modern standards. (credit: Chemical Heritage Foundation)

From alchemy came the historical progressions that led to modern chemistry: the isolation of drugs from natural sources, metallurgy, and the dye industry. Today, chemistry continues to deepen our understanding and improve our ability to harness and control the behavior of matter.

# Chemistry: The Central Science 

Chemistry is sometimes referred to as "the central science" due to its interconnectedness with a vast array of other STEM disciplines (STEM stands for areas of study in the science, technology, engineering, and math fields). Chemistry and the language of chemists play vital roles in biology, medicine, materials science, forensics, environmental science, and many other fields (Figure 1.3). The basic principles of physics are essential for understanding many aspects of chemistry, and there is extensive overlap between many subdisciplines within the two fields, such as chemical physics and nuclear chemistry. Mathematics, computer science, and information theory provide important tools that help us calculate, interpret, describe, and generally make sense of the chemical world. Biology and chemistry converge in biochemistry, which is crucial to understanding the many complex factors and processes that keep living organisms (such as us) alive. Chemical engineering, materials science, and nanotechnology combine chemical principles and empirical findings to produce useful substances, ranging from gasoline to fabrics to electronics. Agriculture, food science, veterinary science, and brewing and wine making help provide sustenance in the form of food and drink to the world's population. Medicine, pharmacology, biotechnology, and botany identify and produce substances that help keep us healthy. Environmental science, geology, oceanography, and atmospheric science incorporate many chemical ideas to help us better understand and protect our physical world. Chemical ideas are used to help understand the universe in astronomy and cosmology.

![Image](Chapter_1_images/img-2.jpeg)

Figure 1.3 Knowledge of chemistry is central to understanding a wide range of scientific disciplines. This diagram shows just some of the interrelationships between chemistry and other fields.

What are some changes in matter that are essential to daily life? Digesting and assimilating food, synthesizing polymers that are used to make clothing, containers, cookware, and credit cards, and refining crude oil into gasoline and other products are just a few examples. As you proceed through this course, you will discover many different examples of changes in the composition and structure of matter, how to classify these changes and how they occurred, their causes, the changes in energy that accompany them, and the principles and laws involved. As you learn about these things, you will be learning chemistry, the study of the composition, properties, and interactions of matter. The practice of chemistry is not limited to chemistry books or laboratories: It happens whenever someone is involved in changes in matter or in conditions that may lead to such changes.

# The Scientific Method 

Chemistry is a science based on observation and experimentation. Doing chemistry involves attempting to answer questions and explain observations in terms of the laws and theories of chemistry, using procedures that are accepted by the scientific community. There is no single route to answering a question or explaining an observation, but there is an aspect common to every approach: Each uses knowledge based on experiments that can be reproduced to verify the results. Some routes involve a hypothesis, a tentative explanation of observations that acts as a guide for gathering and checking information. A hypothesis is tested by experimentation, calculation, and/or comparison with the experiments of others and then refined as needed.

Some hypotheses are attempts to explain the behavior that is summarized in laws. The laws of science summarize a vast number of experimental observations, and describe or predict some facet of the natural world. If such a hypothesis turns out to be capable of explaining a large body of experimental data, it can reach the status of a theory. Scientific theories are well-substantiated, comprehensive, testable explanations of particular aspects of nature. Theories are accepted because they provide satisfactory explanations, but they can be modified if new data become available. The path of discovery that leads from question and observation to law or hypothesis to theory, combined with experimental verification of the hypothesis and any necessary modification of the theory, is called the scientific method (Figure 1.4).

![Image](Chapter_1_images/img-3.jpeg)

Figure 1.4 The scientific method follows a process similar to the one shown in this diagram. All the key components are shown, in roughly the right order. Scientific progress is seldom neat and clean: It requires open inquiry and the reworking of questions and ideas in response to findings.

# The Domains of Chemistry 

Chemists study and describe the behavior of matter and energy in three different domains: macroscopic, microscopic, and symbolic. These domains provide different ways of considering and describing chemical behavior.

Macro is a Greek word that means "large." The macroscopic domain is familiar to us: It is the realm of everyday things that are large enough to be sensed directly by human sight or touch. In daily life, this includes the food you eat and the breeze you feel on your face. The macroscopic domain includes everyday and laboratory chemistry, where we observe and measure physical and chemical properties such as density, solubility, and flammability.

Micro comes from Greek and means "small." The microscopic domain of chemistry is often visited in the imagination. Some aspects of the microscopic domain are visible through standard optical microscopes, for example, many biological cells. More sophisticated instruments are capable of imaging even smaller entities such as molecules and atoms (see Figure $1.5(\mathbf{b})$ ).

However, most of the subjects in the microscopic domain of chemistry are too small to be seen even with the most advanced microscopes and may only be pictured in the mind. Other components of the microscopic domain include ions and electrons, protons and neutrons, and chemical bonds, each of which is far too small to see.

The symbolic domain contains the specialized language used to represent components of the macroscopic and microscopic domains. Chemical symbols (such as those used in the periodic table), chemical formulas, and chemical equations are part of the symbolic domain, as are graphs, drawings, and calculations. These symbols play an important role in chemistry because they help interpret the behavior of the macroscopic domain in terms of the components of the microscopic domain. One of the challenges for students learning chemistry is recognizing that the same symbols can represent different things in the macroscopic and microscopic domains, and one of the features that makes chemistry fascinating is the use of a domain that must be imagined to explain behavior in a domain that can be observed.

A helpful way to understand the three domains is via the essential and ubiquitous substance of water. That water is a liquid at moderate temperatures, will freeze to form a solid at lower temperatures, and boil to form a gas at higher temperatures (Figure 1.5) are macroscopic observations. But some properties of water fall into the microscopic domain-what cannot be observed with the naked eye. The description of water as comprising two hydrogen atoms and one oxygen atom, and the explanation of freezing and boiling in terms of attractions between these molecules, is within the microscopic arena. The formula $\mathrm{H}_{2} \mathrm{O}$, which can describe water at either the macroscopic or microscopic levels, is an example of the symbolic domain. The abbreviations $(g)$ for gas, $(s)$ for solid, and $(l)$ for liquid are also symbolic.
![Image](Chapter_1_images/img-4.jpeg)

Figure 1.5 (a) Moisture in the air, icebergs, and the ocean represent water in the macroscopic domain. (b) At the molecular level (microscopic domain), gas molecules are far apart and disorganized, solid water molecules are close together and organized, and liquid molecules are close together and disorganized. (c) The formula $\mathrm{H}_{2} \mathrm{O}$ symbolizes water, and $(g),(s)$, and $(l)$ symbolize its phases. Note that clouds actually comprise either very small liquid water droplets or solid water crystals; gaseous water in our atmosphere is not visible to the naked eye, although it may be sensed as humidity. (credit a: modification of work by "Gorkaazk"/Wikimedia Commons)

# 1.2 Phases and Classification of Matter 

By the end of this section, you will be able to:

- Describe the basic properties of each physical state of matter: solid, liquid, and gas
- Distinguish between mass and weight
- Apply the law of conservation of matter
- Classify matter as an element, compound, homogeneous mixture, or heterogeneous mixture with regard to its physical state and composition
- Define and give examples of atoms and molecules

Matter is defined as anything that occupies space and has mass, and it is all around us. Solids and liquids are more obviously matter: We can see that they take up space, and their weight tells us that they have mass. Gases are also

matter; if gases did not take up space, a balloon would not inflate (increase its volume) when filled with gas.
Solids, liquids, and gases are the three states of matter commonly found on earth (Figure 1.6). A solid is rigid and possesses a definite shape. A liquid flows and takes the shape of its container, except that it forms a flat or slightly curved upper surface when acted upon by gravity. (In zero gravity, liquids assume a spherical shape.) Both liquid and solid samples have volumes that are very nearly independent of pressure. A gas takes both the shape and volume of its container.
![Image](Chapter_1_images/img-5.jpeg)

Figure 1.6 The three most common states or phases of matter are solid, liquid, and gas.

A fourth state of matter, plasma, occurs naturally in the interiors of stars. A plasma is a gaseous state of matter that contains appreciable numbers of electrically charged particles (Figure 1.7). The presence of these charged particles imparts unique properties to plasmas that justify their classification as a state of matter distinct from gases. In addition to stars, plasmas are found in some other high-temperature environments (both natural and man-made), such as lightning strikes, certain television screens, and specialized analytical instruments used to detect trace amounts of metals.

![Image](Chapter_1_images/img-6.jpeg)

Figure 1.7 A plasma torch can be used to cut metal. (credit: "Hypertherm"/Wikimedia Commons)

# Link to Learning 

In a tiny cell in a plasma television, the plasma emits ultraviolet light, which in turn causes the display at that location to appear a specific color. The composite of these tiny dots of color makes up the image that you see. Watch this video (http://openstaxcollege.org/1/16plasma) to learn more about plasma and the places you encounter it.

Some samples of matter appear to have properties of solids, liquids, and/or gases at the same time. This can occur when the sample is composed of many small pieces. For example, we can pour sand as if it were a liquid because it is composed of many small grains of solid sand. Matter can also have properties of more than one state when it is a mixture, such as with clouds. Clouds appear to behave somewhat like gases, but they are actually mixtures of air (gas) and tiny particles of water (liquid or solid).

The mass of an object is a measure of the amount of matter in it. One way to measure an object's mass is to measure the force it takes to accelerate the object. It takes much more force to accelerate a car than a bicycle because the car has much more mass. A more common way to determine the mass of an object is to use a balance to compare its mass with a standard mass.

Although weight is related to mass, it is not the same thing. Weight refers to the force that gravity exerts on an object. This force is directly proportional to the mass of the object. The weight of an object changes as the force of gravity changes, but its mass does not. An astronaut's mass does not change just because she goes to the moon. But her weight on the moon is only one-sixth her earth-bound weight because the moon's gravity is only one-sixth that of the earth's. She may feel "weightless" during her trip when she experiences negligible external forces (gravitational or any other), although she is, of course, never "massless."

The law of conservation of matter summarizes many scientific observations about matter: It states that there is no detectable change in the total quantity of matter present when matter converts from one type to another (a chemical change) or changes among solid, liquid, or gaseous states (a physical change). Brewing beer and the operation of batteries provide examples of the conservation of matter (Figure 1.8). During the brewing of beer, the ingredients (water, yeast, grains, malt, hops, and sugar) are converted into beer (water, alcohol, carbonation, and flavoring substances) with no actual loss of substance. This is most clearly seen during the bottling process, when glucose turns into ethanol and carbon dioxide, and the total mass of the substances does not change. This can also be seen in a

lead-acid car battery: The original substances (lead, lead oxide, and sulfuric acid), which are capable of producing electricity, are changed into other substances (lead sulfate and water) that do not produce electricity, with no change in the actual amount of matter.
![Image](Chapter_1_images/img-7.jpeg)

Figure 1.8 (a) The mass of beer precursor materials is the same as the mass of beer produced: Sugar has become alcohol and carbon dioxide. (b) The mass of the lead, lead oxide, and sulfuric acid consumed by the production of electricity is exactly equal to the mass of lead sulfate and water that is formed.

Although this conservation law holds true for all conversions of matter, convincing examples are few and far between because, outside of the controlled conditions in a laboratory, we seldom collect all of the material that is produced during a particular conversion. For example, when you eat, digest, and assimilate food, all of the matter in the original food is preserved. But because some of the matter is incorporated into your body, and much is excreted as various types of waste, it is challenging to verify by measurement.

# Classifying Matter 

Matter can be classified into several categories. Two broad categories are mixtures and pure substances. A pure substance has a constant composition. All specimens of a pure substance have exactly the same makeup and properties. Any sample of sucrose (table sugar) consists of $42.1 \%$ carbon, $6.5 \%$ hydrogen, and $51.4 \%$ oxygen by mass. Any sample of sucrose also has the same physical properties, such as melting point, color, and sweetness, regardless of the source from which it is isolated.

Pure substances may be divided into two classes: elements and compounds. Pure substances that cannot be broken down into simpler substances by chemical changes are called elements. Iron, silver, gold, aluminum, sulfur, oxygen, and copper are familiar examples of the more than 100 known elements, of which about 90 occur naturally on the earth, and two dozen or so have been created in laboratories.

Pure substances that can be broken down by chemical changes are called compounds. This breakdown may produce either elements or other compounds, or both. Mercury(II) oxide, an orange, crystalline solid, can be broken down by heat into the elements mercury and oxygen (Figure 1.9). When heated in the absence of air, the compound sucrose is broken down into the element carbon and the compound water. (The initial stage of this process, when the sugar is turning brown, is known as caramelization-this is what imparts the characteristic sweet and nutty flavor to caramel apples, caramelized onions, and caramel). Silver(I) chloride is a white solid that can be broken down into its elements, silver and chlorine, by absorption of light. This property is the basis for the use of this compound in photographic

films and photochromic eyeglasses (those with lenses that darken when exposed to light).
![Image](Chapter_1_images/img-8.jpeg)
(a)
![Image](Chapter_1_images/img-9.jpeg)
(b)
![Image](Chapter_1_images/img-10.jpeg)
(c)

Figure 1.9 (a) The compound mercury(II) oxide, (b) when heated, (c) decomposes into silvery droplets of liquid mercury and invisible oxygen gas. (credit: modification of work by Paul Flowers)

# Link to Learning 

Many compounds break down when heated. This site (http://openstaxcollege.org/1/16mercury) shows the breakdown of mercury oxide, HgO . You can also view an example of the photochemical decomposition of silver chloride (http://openstaxcollege.org/1/16silvchloride) (AgCl), the basis of early photography.

The properties of combined elements are different from those in the free, or uncombined, state. For example, white crystalline sugar (sucrose) is a compound resulting from the chemical combination of the element carbon, which is a black solid in one of its uncombined forms, and the two elements hydrogen and oxygen, which are colorless gases when uncombined. Free sodium, an element that is a soft, shiny, metallic solid, and free chlorine, an element that is a yellow-green gas, combine to form sodium chloride (table salt), a compound that is a white, crystalline solid.

A mixture is composed of two or more types of matter that can be present in varying amounts and can be separated by physical changes, such as evaporation (you will learn more about this later). A mixture with a composition that varies from point to point is called a heterogeneous mixture. Italian dressing is an example of a heterogeneous mixture (Figure 1.10). Its composition can vary because it may be prepared from varying amounts of oil, vinegar, and herbs. It is not the same from point to point throughout the mixture-one drop may be mostly vinegar, whereas a different drop may be mostly oil or herbs because the oil and vinegar separate and the herbs settle. Other examples of heterogeneous mixtures are chocolate chip cookies (we can see the separate bits of chocolate, nuts, and cookie dough) and granite (we can see the quartz, mica, feldspar, and more).

A homogeneous mixture, also called a solution, exhibits a uniform composition and appears visually the same throughout. An example of a solution is a sports drink, consisting of water, sugar, coloring, flavoring, and electrolytes mixed together uniformly (Figure 1.10). Each drop of a sports drink tastes the same because each drop contains the same amounts of water, sugar, and other components. Note that the composition of a sports drink can vary-it could be made with somewhat more or less sugar, flavoring, or other components, and still be a sports drink. Other examples of homogeneous mixtures include air, maple syrup, gasoline, and a solution of salt in water.

![Image](Chapter_1_images/img-11.jpeg)

Figure 1.10 (a) Oil and vinegar salad dressing is a heterogeneous mixture because its composition is not uniform throughout. (b) A commercial sports drink is a homogeneous mixture because its composition is uniform throughout. (credit a "left": modification of work by John Mayer; credit a "right": modification of work by Umberto Salvagnin; credit b "left: modification of work by Jeff Bedford)

Although there are just over 100 elements, tens of millions of chemical compounds result from different combinations of these elements. Each compound has a specific composition and possesses definite chemical and physical properties that distinguish it from all other compounds. And, of course, there are innumerable ways to combine elements and compounds to form different mixtures. A summary of how to distinguish between the various major classifications of matter is shown in (Figure 1.11).
![Image](Chapter_1_images/img-12.jpeg)

Figure 1.11 Depending on its properties, a given substance can be classified as a homogeneous mixture, a heterogeneous mixture, a compound, or an element.

Eleven elements make up about 99\% of the earth's crust and atmosphere (Table 1.1). Oxygen constitutes nearly onehalf and silicon about one-quarter of the total quantity of these elements. A majority of elements on earth are found in chemical combinations with other elements; about one-quarter of the elements are also found in the free state.

Elemental Composition of Earth

| Element | Symbol | Percent Mass |  | Element | Symbol | Percent Mass |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| oxygen | O | 49.20 |  | chlorine | Cl | 0.19 |
| silicon | Si | 25.67 |  | phosphorus | P | 0.11 |
| aluminum | Al | 7.50 |  | manganese | Mn | 0.09 |

Table 1.1

# Chapter 1 - Essential Ideas

![Image](Chapter_1_images/img-0.jpeg)

Figure 1.1 Chemical substances and processes are essential for our existence, providing sustenance, keeping us clean and healthy, fabricating electronic devices, enabling transportation, and much more. (credit "left": modification of work by "vxla"/Flickr; credit "left middle": modification of work by "the Italian voice"/Flickr; credit "right middle": modification of work by Jason Trim; credit "right": modification of work by "gosheshe"/Flickr)

## Chapter Outline

1.1 Chemistry in Context
1.2 Phases and Classification of Matter
1.3 Physical and Chemical Properties
1.4 Measurements
1.5 Measurement Uncertainty, Accuracy, and Precision
1.6 Mathematical Treatment of Measurement Results

## Introduction

Your alarm goes off and, after hitting "snooze" once or twice, you pry yourself out of bed. You make a cup of coffee to help you get going, and then you shower, get dressed, eat breakfast, and check your phone for messages. On your way to school, you stop to fill your car's gas tank, almost making you late for the first day of chemistry class. As you find a seat in the classroom, you read the question projected on the screen: "Welcome to class! Why should we study chemistry?"

Do you have an answer? You may be studying chemistry because it fulfills an academic requirement, but if you consider your daily activities, you might find chemistry interesting for other reasons. Most everything you do and encounter during your day involves chemistry. Making coffee, cooking eggs, and toasting bread involve chemistry. The products you use-like soap and shampoo, the fabrics you wear, the electronics that keep you connected to your world, the gasoline that propels your car-all of these and more involve chemical substances and processes. Whether you are aware or not, chemistry is part of your everyday world. In this course, you will learn many of the essential principles underlying the chemistry of modern-day life.

# 1.1 Chemistry in Context 

By the end of this module, you will be able to:

- Outline the historical development of chemistry
- Provide examples of the importance of chemistry in everyday life
- Describe the scientific method
- Differentiate among hypotheses, theories, and laws
- Provide examples illustrating macroscopic, microscopic, and symbolic domains

Throughout human history, people have tried to convert matter into more useful forms. Our Stone Age ancestors chipped pieces of flint into useful tools and carved wood into statues and toys. These endeavors involved changing the shape of a substance without changing the substance itself. But as our knowledge increased, humans began to change the composition of the substances as well-clay was converted into pottery, hides were cured to make garments, copper ores were transformed into copper tools and weapons, and grain was made into bread.
Humans began to practice chemistry when they learned to control fire and use it to cook, make pottery, and smelt metals. Subsequently, they began to separate and use specific components of matter. A variety of drugs such as aloe, myrrh, and opium were isolated from plants. Dyes, such as indigo and Tyrian purple, were extracted from plant and animal matter. Metals were combined to form alloys-for example, copper and tin were mixed together to make bronze-and more elaborate smelting techniques produced iron. Alkalis were extracted from ashes, and soaps were prepared by combining these alkalis with fats. Alcohol was produced by fermentation and purified by distillation.
Attempts to understand the behavior of matter extend back for more than 2500 years. As early as the sixth century BC, Greek philosophers discussed a system in which water was the basis of all things. You may have heard of the Greek postulate that matter consists of four elements: earth, air, fire, and water. Subsequently, an amalgamation of chemical technologies and philosophical speculations was spread from Egypt, China, and the eastern Mediterranean by alchemists, who endeavored to transform "base metals" such as lead into "noble metals" like gold, and to create elixirs to cure disease and extend life (Figure 1.2).

![Image](Chapter_1_images/img-1.jpeg)

Figure 1.2 This portrayal shows an alchemist's workshop circa 1580. Although alchemy made some useful contributions to how to manipulate matter, it was not scientific by modern standards. (credit: Chemical Heritage Foundation)

From alchemy came the historical progressions that led to modern chemistry: the isolation of drugs from natural sources, metallurgy, and the dye industry. Today, chemistry continues to deepen our understanding and improve our ability to harness and control the behavior of matter.

# Chemistry: The Central Science 

Chemistry is sometimes referred to as "the central science" due to its interconnectedness with a vast array of other STEM disciplines (STEM stands for areas of study in the science, technology, engineering, and math fields). Chemistry and the language of chemists play vital roles in biology, medicine, materials science, forensics, environmental science, and many other fields (Figure 1.3). The basic principles of physics are essential for understanding many aspects of chemistry, and there is extensive overlap between many subdisciplines within the two fields, such as chemical physics and nuclear chemistry. Mathematics, computer science, and information theory provide important tools that help us calculate, interpret, describe, and generally make sense of the chemical world. Biology and chemistry converge in biochemistry, which is crucial to understanding the many complex factors and processes that keep living organisms (such as us) alive. Chemical engineering, materials science, and nanotechnology combine chemical principles and empirical findings to produce useful substances, ranging from gasoline to fabrics to electronics. Agriculture, food science, veterinary science, and brewing and wine making help provide sustenance in the form of food and drink to the world's population. Medicine, pharmacology, biotechnology, and botany identify and produce substances that help keep us healthy. Environmental science, geology, oceanography, and atmospheric science incorporate many chemical ideas to help us better understand and protect our physical world. Chemical ideas are used to help understand the universe in astronomy and cosmology.

![Image](Chapter_1_images/img-2.jpeg)

Figure 1.3 Knowledge of chemistry is central to understanding a wide range of scientific disciplines. This diagram shows just some of the interrelationships between chemistry and other fields.

What are some changes in matter that are essential to daily life? Digesting and assimilating food, synthesizing polymers that are used to make clothing, containers, cookware, and credit cards, and refining crude oil into gasoline and other products are just a few examples. As you proceed through this course, you will discover many different examples of changes in the composition and structure of matter, how to classify these changes and how they occurred, their causes, the changes in energy that accompany them, and the principles and laws involved. As you learn about these things, you will be learning chemistry, the study of the composition, properties, and interactions of matter. The practice of chemistry is not limited to chemistry books or laboratories: It happens whenever someone is involved in changes in matter or in conditions that may lead to such changes.

# The Scientific Method 

Chemistry is a science based on observation and experimentation. Doing chemistry involves attempting to answer questions and explain observations in terms of the laws and theories of chemistry, using procedures that are accepted by the scientific community. There is no single route to answering a question or explaining an observation, but there is an aspect common to every approach: Each uses knowledge based on experiments that can be reproduced to verify the results. Some routes involve a hypothesis, a tentative explanation of observations that acts as a guide for gathering and checking information. A hypothesis is tested by experimentation, calculation, and/or comparison with the experiments of others and then refined as needed.

Some hypotheses are attempts to explain the behavior that is summarized in laws. The laws of science summarize a vast number of experimental observations, and describe or predict some facet of the natural world. If such a hypothesis turns out to be capable of explaining a large body of experimental data, it can reach the status of a theory. Scientific theories are well-substantiated, comprehensive, testable explanations of particular aspects of nature. Theories are accepted because they provide satisfactory explanations, but they can be modified if new data become available. The path of discovery that leads from question and observation to law or hypothesis to theory, combined with experimental verification of the hypothesis and any necessary modification of the theory, is called the scientific method (Figure 1.4).

![Image](Chapter_1_images/img-3.jpeg)

Figure 1.4 The scientific method follows a process similar to the one shown in this diagram. All the key components are shown, in roughly the right order. Scientific progress is seldom neat and clean: It requires open inquiry and the reworking of questions and ideas in response to findings.

# The Domains of Chemistry 

Chemists study and describe the behavior of matter and energy in three different domains: macroscopic, microscopic, and symbolic. These domains provide different ways of considering and describing chemical behavior.

Macro is a Greek word that means "large." The macroscopic domain is familiar to us: It is the realm of everyday things that are large enough to be sensed directly by human sight or touch. In daily life, this includes the food you eat and the breeze you feel on your face. The macroscopic domain includes everyday and laboratory chemistry, where we observe and measure physical and chemical properties such as density, solubility, and flammability.

Micro comes from Greek and means "small." The microscopic domain of chemistry is often visited in the imagination. Some aspects of the microscopic domain are visible through standard optical microscopes, for example, many biological cells. More sophisticated instruments are capable of imaging even smaller entities such as molecules and atoms (see Figure $1.5(\mathbf{b})$ ).

However, most of the subjects in the microscopic domain of chemistry are too small to be seen even with the most advanced microscopes and may only be pictured in the mind. Other components of the microscopic domain include ions and electrons, protons and neutrons, and chemical bonds, each of which is far too small to see.

The symbolic domain contains the specialized language used to represent components of the macroscopic and microscopic domains. Chemical symbols (such as those used in the periodic table), chemical formulas, and chemical equations are part of the symbolic domain, as are graphs, drawings, and calculations. These symbols play an important role in chemistry because they help interpret the behavior of the macroscopic domain in terms of the components of the microscopic domain. One of the challenges for students learning chemistry is recognizing that the same symbols can represent different things in the macroscopic and microscopic domains, and one of the features that makes chemistry fascinating is the use of a domain that must be imagined to explain behavior in a domain that can be observed.

A helpful way to understand the three domains is via the essential and ubiquitous substance of water. That water is a liquid at moderate temperatures, will freeze to form a solid at lower temperatures, and boil to form a gas at higher temperatures (Figure 1.5) are macroscopic observations. But some properties of water fall into the microscopic domain-what cannot be observed with the naked eye. The description of water as comprising two hydrogen atoms and one oxygen atom, and the explanation of freezing and boiling in terms of attractions between these molecules, is within the microscopic arena. The formula $\mathrm{H}_{2} \mathrm{O}$, which can describe water at either the macroscopic or microscopic levels, is an example of the symbolic domain. The abbreviations $(g)$ for gas, $(s)$ for solid, and $(l)$ for liquid are also symbolic.
![Image](Chapter_1_images/img-4.jpeg)

Figure 1.5 (a) Moisture in the air, icebergs, and the ocean represent water in the macroscopic domain. (b) At the molecular level (microscopic domain), gas molecules are far apart and disorganized, solid water molecules are close together and organized, and liquid molecules are close together and disorganized. (c) The formula $\mathrm{H}_{2} \mathrm{O}$ symbolizes water, and $(g),(s)$, and $(l)$ symbolize its phases. Note that clouds actually comprise either very small liquid water droplets or solid water crystals; gaseous water in our atmosphere is not visible to the naked eye, although it may be sensed as humidity. (credit a: modification of work by "Gorkaazk"/Wikimedia Commons)

# 1.2 Phases and Classification of Matter 

By the end of this section, you will be able to:

- Describe the basic properties of each physical state of matter: solid, liquid, and gas
- Distinguish between mass and weight
- Apply the law of conservation of matter
- Classify matter as an element, compound, homogeneous mixture, or heterogeneous mixture with regard to its physical state and composition
- Define and give examples of atoms and molecules

Matter is defined as anything that occupies space and has mass, and it is all around us. Solids and liquids are more obviously matter: We can see that they take up space, and their weight tells us that they have mass. Gases are also

matter; if gases did not take up space, a balloon would not inflate (increase its volume) when filled with gas.
Solids, liquids, and gases are the three states of matter commonly found on earth (Figure 1.6). A solid is rigid and possesses a definite shape. A liquid flows and takes the shape of its container, except that it forms a flat or slightly curved upper surface when acted upon by gravity. (In zero gravity, liquids assume a spherical shape.) Both liquid and solid samples have volumes that are very nearly independent of pressure. A gas takes both the shape and volume of its container.
![Image](Chapter_1_images/img-5.jpeg)

Figure 1.6 The three most common states or phases of matter are solid, liquid, and gas.

A fourth state of matter, plasma, occurs naturally in the interiors of stars. A plasma is a gaseous state of matter that contains appreciable numbers of electrically charged particles (Figure 1.7). The presence of these charged particles imparts unique properties to plasmas that justify their classification as a state of matter distinct from gases. In addition to stars, plasmas are found in some other high-temperature environments (both natural and man-made), such as lightning strikes, certain television screens, and specialized analytical instruments used to detect trace amounts of metals.

![Image](Chapter_1_images/img-6.jpeg)

Figure 1.7 A plasma torch can be used to cut metal. (credit: "Hypertherm"/Wikimedia Commons)

# Link to Learning 

In a tiny cell in a plasma television, the plasma emits ultraviolet light, which in turn causes the display at that location to appear a specific color. The composite of these tiny dots of color makes up the image that you see. Watch this video (http://openstaxcollege.org/1/16plasma) to learn more about plasma and the places you encounter it.

Some samples of matter appear to have properties of solids, liquids, and/or gases at the same time. This can occur when the sample is composed of many small pieces. For example, we can pour sand as if it were a liquid because it is composed of many small grains of solid sand. Matter can also have properties of more than one state when it is a mixture, such as with clouds. Clouds appear to behave somewhat like gases, but they are actually mixtures of air (gas) and tiny particles of water (liquid or solid).

The mass of an object is a measure of the amount of matter in it. One way to measure an object's mass is to measure the force it takes to accelerate the object. It takes much more force to accelerate a car than a bicycle because the car has much more mass. A more common way to determine the mass of an object is to use a balance to compare its mass with a standard mass.

Although weight is related to mass, it is not the same thing. Weight refers to the force that gravity exerts on an object. This force is directly proportional to the mass of the object. The weight of an object changes as the force of gravity changes, but its mass does not. An astronaut's mass does not change just because she goes to the moon. But her weight on the moon is only one-sixth her earth-bound weight because the moon's gravity is only one-sixth that of the earth's. She may feel "weightless" during her trip when she experiences negligible external forces (gravitational or any other), although she is, of course, never "massless."

The law of conservation of matter summarizes many scientific observations about matter: It states that there is no detectable change in the total quantity of matter present when matter converts from one type to another (a chemical change) or changes among solid, liquid, or gaseous states (a physical change). Brewing beer and the operation of batteries provide examples of the conservation of matter (Figure 1.8). During the brewing of beer, the ingredients (water, yeast, grains, malt, hops, and sugar) are converted into beer (water, alcohol, carbonation, and flavoring substances) with no actual loss of substance. This is most clearly seen during the bottling process, when glucose turns into ethanol and carbon dioxide, and the total mass of the substances does not change. This can also be seen in a

lead-acid car battery: The original substances (lead, lead oxide, and sulfuric acid), which are capable of producing electricity, are changed into other substances (lead sulfate and water) that do not produce electricity, with no change in the actual amount of matter.
![Image](Chapter_1_images/img-7.jpeg)

Figure 1.8 (a) The mass of beer precursor materials is the same as the mass of beer produced: Sugar has become alcohol and carbon dioxide. (b) The mass of the lead, lead oxide, and sulfuric acid consumed by the production of electricity is exactly equal to the mass of lead sulfate and water that is formed.

Although this conservation law holds true for all conversions of matter, convincing examples are few and far between because, outside of the controlled conditions in a laboratory, we seldom collect all of the material that is produced during a particular conversion. For example, when you eat, digest, and assimilate food, all of the matter in the original food is preserved. But because some of the matter is incorporated into your body, and much is excreted as various types of waste, it is challenging to verify by measurement.

# Classifying Matter 

Matter can be classified into several categories. Two broad categories are mixtures and pure substances. A pure substance has a constant composition. All specimens of a pure substance have exactly the same makeup and properties. Any sample of sucrose (table sugar) consists of $42.1 \%$ carbon, $6.5 \%$ hydrogen, and $51.4 \%$ oxygen by mass. Any sample of sucrose also has the same physical properties, such as melting point, color, and sweetness, regardless of the source from which it is isolated.

Pure substances may be divided into two classes: elements and compounds. Pure substances that cannot be broken down into simpler substances by chemical changes are called elements. Iron, silver, gold, aluminum, sulfur, oxygen, and copper are familiar examples of the more than 100 known elements, of which about 90 occur naturally on the earth, and two dozen or so have been created in laboratories.

Pure substances that can be broken down by chemical changes are called compounds. This breakdown may produce either elements or other compounds, or both. Mercury(II) oxide, an orange, crystalline solid, can be broken down by heat into the elements mercury and oxygen (Figure 1.9). When heated in the absence of air, the compound sucrose is broken down into the element carbon and the compound water. (The initial stage of this process, when the sugar is turning brown, is known as caramelization-this is what imparts the characteristic sweet and nutty flavor to caramel apples, caramelized onions, and caramel). Silver(I) chloride is a white solid that can be broken down into its elements, silver and chlorine, by absorption of light. This property is the basis for the use of this compound in photographic

films and photochromic eyeglasses (those with lenses that darken when exposed to light).
![Image](Chapter_1_images/img-8.jpeg)
(a)
![Image](Chapter_1_images/img-9.jpeg)
(b)
![Image](Chapter_1_images/img-10.jpeg)
(c)

Figure 1.9 (a) The compound mercury(II) oxide, (b) when heated, (c) decomposes into silvery droplets of liquid mercury and invisible oxygen gas. (credit: modification of work by Paul Flowers)

# Link to Learning 

Many compounds break down when heated. This site (http://openstaxcollege.org/1/16mercury) shows the breakdown of mercury oxide, HgO . You can also view an example of the photochemical decomposition of silver chloride (http://openstaxcollege.org/1/16silvchloride) (AgCl), the basis of early photography.

The properties of combined elements are different from those in the free, or uncombined, state. For example, white crystalline sugar (sucrose) is a compound resulting from the chemical combination of the element carbon, which is a black solid in one of its uncombined forms, and the two elements hydrogen and oxygen, which are colorless gases when uncombined. Free sodium, an element that is a soft, shiny, metallic solid, and free chlorine, an element that is a yellow-green gas, combine to form sodium chloride (table salt), a compound that is a white, crystalline solid.

A mixture is composed of two or more types of matter that can be present in varying amounts and can be separated by physical changes, such as evaporation (you will learn more about this later). A mixture with a composition that varies from point to point is called a heterogeneous mixture. Italian dressing is an example of a heterogeneous mixture (Figure 1.10). Its composition can vary because it may be prepared from varying amounts of oil, vinegar, and herbs. It is not the same from point to point throughout the mixture-one drop may be mostly vinegar, whereas a different drop may be mostly oil or herbs because the oil and vinegar separate and the herbs settle. Other examples of heterogeneous mixtures are chocolate chip cookies (we can see the separate bits of chocolate, nuts, and cookie dough) and granite (we can see the quartz, mica, feldspar, and more).

A homogeneous mixture, also called a solution, exhibits a uniform composition and appears visually the same throughout. An example of a solution is a sports drink, consisting of water, sugar, coloring, flavoring, and electrolytes mixed together uniformly (Figure 1.10). Each drop of a sports drink tastes the same because each drop contains the same amounts of water, sugar, and other components. Note that the composition of a sports drink can vary-it could be made with somewhat more or less sugar, flavoring, or other components, and still be a sports drink. Other examples of homogeneous mixtures include air, maple syrup, gasoline, and a solution of salt in water.

![Image](Chapter_1_images/img-11.jpeg)

Figure 1.10 (a) Oil and vinegar salad dressing is a heterogeneous mixture because its composition is not uniform throughout. (b) A commercial sports drink is a homogeneous mixture because its composition is uniform throughout. (credit a "left": modification of work by John Mayer; credit a "right": modification of work by Umberto Salvagnin; credit b "left: modification of work by Jeff Bedford)

Although there are just over 100 elements, tens of millions of chemical compounds result from different combinations of these elements. Each compound has a specific composition and possesses definite chemical and physical properties that distinguish it from all other compounds. And, of course, there are innumerable ways to combine elements and compounds to form different mixtures. A summary of how to distinguish between the various major classifications of matter is shown in (Figure 1.11).
![Image](Chapter_1_images/img-12.jpeg)

Figure 1.11 Depending on its properties, a given substance can be classified as a homogeneous mixture, a heterogeneous mixture, a compound, or an element.

Eleven elements make up about 99\% of the earth's crust and atmosphere (Table 1.1). Oxygen constitutes nearly onehalf and silicon about one-quarter of the total quantity of these elements. A majority of elements on earth are found in chemical combinations with other elements; about one-quarter of the elements are also found in the free state.

Elemental Composition of Earth

| Element | Symbol | Percent Mass |  | Element | Symbol | Percent Mass |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| oxygen | O | 49.20 |  | chlorine | Cl | 0.19 |
| silicon | Si | 25.67 |  | phosphorus | P | 0.11 |
| aluminum | Al | 7.50 |  | manganese | Mn | 0.09 |

Table 1.1

Elemental Composition of Earth

| Element | Symbol | Percent Mass |  | Element | Symbol | Percent Mass |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| iron | Fe | 4.71 |  | carbon | C | 0.08 |
| calcium | Ca | 3.39 |  | sulfur | S | 0.06 |
| sodium | Na | 2.63 |  | barium | Ba | 0.04 |
| potassium | K | 2.40 |  | nitrogen | N | 0.03 |
| magnesium | Mg | 1.93 |  | fluorine | F | 0.03 |
| hydrogen | H | 0.87 |  | strontium | Sr | 0.02 |
| titanium | Ti | 0.58 |  | all others | - | 0.47 |

Table 1.1

# Atoms and Molecules 

An atom is the smallest particle of an element that has the properties of that element and can enter into a chemical combination. Consider the element gold, for example. Imagine cutting a gold nugget in half, then cutting one of the halves in half, and repeating this process until a piece of gold remained that was so small that it could not be cut in half (regardless of how tiny your knife may be). This minimally sized piece of gold is an atom (from the Greek atomos, meaning "indivisible") (Figure 1.12). This atom would no longer be gold if it were divided any further.
![Image](Chapter_1_images/img-13.jpeg)

Figure 1.12 (a) This photograph shows a gold nugget. (b) A scanning-tunneling microscope (STM) can generate views of the surfaces of solids, such as this image of a gold crystal. Each sphere represents one gold atom. (credit a: modification of work by United States Geological Survey; credit b: modification of work by "Erwinrossen"/Wikimedia Commons)

The first suggestion that matter is composed of atoms is attributed to the Greek philosophers Leucippus and Democritus, who developed their ideas in the 5th century BCE. However, it was not until the early nineteenth century that John Dalton (1766-1844), a British schoolteacher with a keen interest in science, supported this hypothesis with quantitative measurements. Since that time, repeated experiments have confirmed many aspects of this hypothesis, and it has become one of the central theories of chemistry. Other aspects of Dalton's atomic theory are still used but with minor revisions (details of Dalton's theory are provided in the chapter on atoms and molecules).

An atom is so small that its size is difficult to imagine. One of the smallest things we can see with our unaided eye is a single thread of a spider web: These strands are about $1 / 10,000$ of a centimeter $(0.0001 \mathrm{~cm})$ in diameter. Although the cross-section of one strand is almost impossible to see without a microscope, it is huge on an atomic scale. A single carbon atom in the web has a diameter of about 0.000000015 centimeter, and it would take about 7000 carbon atoms to span the diameter of the strand. To put this in perspective, if a carbon atom were the size of a dime, the cross-section of one strand would be larger than a football field, which would require about 150 million carbon atom "dimes" to cover it. (Figure 1.13) shows increasingly close microscopic and atomic-level views of ordinary cotton.
![Image](Chapter_1_images/img-14.jpeg)

Figure 1.13 These images provide an increasingly closer view: (a) a cotton boll, (b) a single cotton fiber viewed under an optical microscope (magnified 40 times), (c) an image of a cotton fiber obtained with an electron microscope (much higher magnification than with the optical microscope); and (d and e) atomic-level models of the fiber (spheres of different colors represent atoms of different elements). (credit c: modification of work by "Featheredtar"/Wikimedia Commons)

An atom is so light that its mass is also difficult to imagine. A billion lead atoms (1,000,000,000 atoms) weigh about $3 \times 10^{-13}$ grams, a mass that is far too light to be weighed on even the world's most sensitive balances. It would require over $300,000,000,000,000$ lead atoms ( 300 trillion, or $3 \times 10^{14}$ ) to be weighed, and they would weigh only 0.0000001 gram.

It is rare to find collections of individual atoms. Only a few elements, such as the gases helium, neon, and argon, consist of a collection of individual atoms that move about independently of one another. Other elements, such as the gases hydrogen, nitrogen, oxygen, and chlorine, are composed of units that consist of pairs of atoms (Figure 1.14). One form of the element phosphorus consists of units composed of four phosphorus atoms. The element sulfur exists in various forms, one of which consists of units composed of eight sulfur atoms. These units are called molecules. A molecule consists of two or more atoms joined by strong forces called chemical bonds. The atoms in a molecule move around as a unit, much like the cans of soda in a six-pack or a bunch of keys joined together on a single key ring. A molecule may consist of two or more identical atoms, as in the molecules found in the elements hydrogen, oxygen, and sulfur, or it may consist of two or more different atoms, as in the molecules found in water. Each water molecule is a unit that contains two hydrogen atoms and one oxygen atom. Each glucose molecule is a unit that contains 6 carbon atoms, 12 hydrogen atoms, and 6 oxygen atoms. Like atoms, molecules are incredibly small and light. If an ordinary glass of water were enlarged to the size of the earth, the water molecules inside it would be about the size of golf balls.

![Image](Chapter_1_images/img-15.jpeg)

Figure 1.14 The elements hydrogen, oxygen, phosphorus, and sulfur form molecules consisting of two or more atoms of the same element. The compounds water, carbon dioxide, and glucose consist of combinations of atoms of different elements.

# Chemistry in Everyday Life 

## Decomposition of Water / Production of Hydrogen

Water consists of the elements hydrogen and oxygen combined in a 2 to 1 ratio. Water can be broken down into hydrogen and oxygen gases by the addition of energy. One way to do this is with a battery or power supply, as shown in (Figure 1.15).

![Image](Chapter_1_images/img-16.jpeg)

Figure 1.15 The decomposition of water is shown at the macroscopic, microscopic, and symbolic levels. The battery provides an electric current (microscopic) that decomposes water. At the macroscopic level, the liquid separates into the gases hydrogen (on the left) and oxygen (on the right). Symbolically, this change is presented by showing how liquid $\mathrm{H}_{2} \mathrm{O}$ separates into $\mathrm{H}_{2}$ and $\mathrm{O}_{2}$ gases.

The breakdown of water involves a rearrangement of the atoms in water molecules into different molecules, each composed of two hydrogen atoms and two oxygen atoms, respectively. Two water molecules form one oxygen molecule and two hydrogen molecules. The representation for what occurs, $2 \mathrm{H}_{2} \mathrm{O}(l) \longrightarrow 2 \mathrm{H}_{2}(g)+\mathrm{O}_{2}(g)$, will be explored in more depth in later chapters.

The two gases produced have distinctly different properties. Oxygen is not flammable but is required for combustion of a fuel, and hydrogen is highly flammable and a potent energy source. How might this knowledge be applied in our world? One application involves research into more fuel-efficient transportation. Fuel-cell vehicles (FCV) run on hydrogen instead of gasoline (Figure 1.16). They are more efficient than vehicles with internal combustion engines, are nonpolluting, and reduce greenhouse gas emissions, making us less dependent on fossil fuels. FCVs are not yet economically viable, however, and current hydrogen production depends on natural gas. If we can develop a process to economically decompose water, or produce hydrogen in another environmentally sound way, FCVs may be the way of the future.

![Image](Chapter_1_images/img-17.jpeg)

Figure 1.16 A fuel cell generates electrical energy from hydrogen and oxygen via an electrochemical process and produces only water as the waste product.

# Chemistry in Everyday Life 

## Chemistry of Cell Phones

Imagine how different your life would be without cell phones (Figure 1.17) and other smart devices. Cell phones are made from numerous chemical substances, which are extracted, refined, purified, and assembled using an extensive and in-depth understanding of chemical principles. About 30\% of the elements that are found in nature are found within a typical smart phone. The case/body/frame consists of a combination of sturdy, durable polymers composed primarily of carbon, hydrogen, oxygen, and nitrogen [acrylonitrile butadiene styrene (ABS) and polycarbonate thermoplastics], and light, strong, structural metals, such as aluminum, magnesium, and iron. The display screen is made from a specially toughened glass (silica glass strengthened by the addition of aluminum, sodium, and potassium) and coated with a material to make it conductive (such as indium tin oxide). The circuit board uses a semiconductor material (usually silicon); commonly used metals like copper, tin, silver, and gold; and more unfamiliar elements such as yttrium, praseodymium, and gadolinium. The battery relies upon lithium ions and a variety of other materials, including iron, cobalt, copper, polyethylene oxide, and polyacrylonitrile.

![Image](Chapter_1_images/img-18.jpeg)

Figure 1.17 Almost one-third of naturally occurring elements are used to make a cell phone. (credit: modification of work by John Taylor)

# 1.3 Physical and Chemical Properties 

By the end of this section, you will be able to:

- Identify properties of and changes in matter as physical or chemical
- Identify properties of matter as extensive or intensive

The characteristics that distinguish one substance from another are called properties. A physical property is a characteristic of matter that is not associated with a change in its chemical composition. Familiar examples of physical properties include density, color, hardness, melting and boiling points, and electrical conductivity. Some physical properties, such as density and color, may be observed without changing the physical state of the matter. Other physical properties, such as the melting temperature of iron or the freezing temperature of water, can only be observed as matter undergoes a physical change. A physical change is a change in the state or properties of matter without any accompanying change in the chemical identities of the substances contained in the matter. Physical changes are observed when wax melts, when sugar dissolves in coffee, and when steam condenses into liquid water (Figure 1.18). Other examples of physical changes include magnetizing and demagnetizing metals (as is done with common antitheft security tags) and grinding solids into powders (which can sometimes yield noticeable changes in color). In each of these examples, there is a change in the physical state, form, or properties of the substance, but no change in its chemical composition.

![Image](Chapter_1_images/img-19.jpeg)

Figure 1.18 (a) Wax undergoes a physical change when solid wax is heated and forms liquid wax. (b) Steam condensing inside a cooking pot is a physical change, as water vapor is changed into liquid water. (credit a: modification of work by "95jb14"/Wikimedia Commons; credit b: modification of work by "mjneuby"/Flickr)

The change of one type of matter into another type (or the inability to change) is a chemical property. Examples of chemical properties include flammability, toxicity, acidity, and many other types of reactivity. Iron, for example, combines with oxygen in the presence of water to form rust; chromium does not oxidize (Figure 1.19). Nitroglycerin is very dangerous because it explodes easily; neon poses almost no hazard because it is very unreactive.
![Image](Chapter_1_images/img-20.jpeg)

Figure 1.19 (a) One of the chemical properties of iron is that it rusts; (b) one of the chemical properties of chromium is that it does not. (credit a: modification of work by Tony Hisgett; credit b: modification of work by "Atoma"/Wikimedia Commons)

A chemical change always produces one or more types of matter that differ from the matter present before the change. The formation of rust is a chemical change because rust is a different kind of matter than the iron, oxygen, and water present before the rust formed. The explosion of nitroglycerin is a chemical change because the gases produced are very different kinds of matter from the original substance. Other examples of chemical changes include reactions that are performed in a lab (such as copper reacting with nitric acid), all forms of combustion (burning), and food being cooked, digested, or rotting (Figure 1.20).

![Image](Chapter_1_images/img-21.jpeg)

Figure 1.20 (a) Copper and nitric acid undergo a chemical change to form copper nitrate and brown, gaseous nitrogen dioxide. (b) During the combustion of a match, cellulose in the match and oxygen from the air undergo a chemical change to form carbon dioxide and water vapor. (c) Cooking red meat causes a number of chemical changes, including the oxidation of iron in myoglobin that results in the familiar red-to-brown color change. (d) A banana turning brown is a chemical change as new, darker (and less tasty) substances form. (credit b: modification of work by Jeff Turner; credit c: modification of work by Gloria Cabada-Leman; credit d: modification of work by Roberto Verzo)

Properties of matter fall into one of two categories. If the property depends on the amount of matter present, it is an extensive property. The mass and volume of a substance are examples of extensive properties; for instance, a gallon of milk has a larger mass than a cup of milk. The value of an extensive property is directly proportional to the amount of matter in question. If the property of a sample of matter does not depend on the amount of matter present, it is an intensive property. Temperature is an example of an intensive property. If the gallon and cup of milk are each at $20^{\circ} \mathrm{C}$ (room temperature), when they are combined, the temperature remains at $20^{\circ} \mathrm{C}$. As another example, consider the distinct but related properties of heat and temperature. A drop of hot cooking oil spattered on your arm causes brief, minor discomfort, whereas a pot of hot oil yields severe burns. Both the drop and the pot of oil are at the same temperature (an intensive property), but the pot clearly contains much more heat (extensive property).

# Chemistry in Everyday Life 

## Hazard Diamond

You may have seen the symbol shown in Figure 1.21 on containers of chemicals in a laboratory or workplace. Sometimes called a "fire diamond" or "hazard diamond," this chemical hazard diamond provides valuable information that briefly summarizes the various dangers of which to be aware when working with a particular

substance.
![Image](Chapter_1_images/img-22.jpeg)

Figure 1.21 The National Fire Protection Agency (NFPA) hazard diamond summarizes the major hazards of a chemical substance.

The National Fire Protection Agency (NFPA) 704 Hazard Identification System was developed by NFPA to provide safety information about certain substances. The system details flammability, reactivity, health, and other hazards. Within the overall diamond symbol, the top (red) diamond specifies the level of fire hazard (temperature range for flash point). The blue (left) diamond indicates the level of health hazard. The yellow (right) diamond describes reactivity hazards, such as how readily the substance will undergo detonation or a violent chemical change. The white (bottom) diamond points out special hazards, such as if it is an oxidizer (which allows the substance to burn in the absence of air/oxygen), undergoes an unusual or dangerous reaction with water, is corrosive, acidic, alkaline, a biological hazard, radioactive, and so on. Each hazard is rated on a scale from 0 to 4 , with 0 being no hazard and 4 being extremely hazardous.

While many elements differ dramatically in their chemical and physical properties, some elements have similar properties. For example, many elements conduct heat and electricity well, whereas others are poor conductors. These properties can be used to sort the elements into three classes: metals (elements that conduct well), nonmetals (elements that conduct poorly), and metalloids (elements that have intermediate conductivities).

The periodic table is a table of elements that places elements with similar properties close together (Figure 1.22). You will learn more about the periodic table as you continue your study of chemistry.

![Image](Chapter_1_images/img-23.jpeg)

Figure 1.22 The periodic table shows how elements may be grouped according to certain similar properties. Note the background color denotes whether an element is a metal, metalloid, or nonmetal, whereas the element symbol color indicates whether it is a solid, liquid, or gas.

# 1.4 Measurements 

By the end of this section, you will be able to:

- Explain the process of measurement
- Identify the three basic parts of a quantity
- Describe the properties and units of length, mass, volume, density, temperature, and time
- Perform basic unit calculations and conversions in the metric and other unit systems

Measurements provide much of the information that informs the hypotheses, theories, and laws describing the behavior of matter and energy in both the macroscopic and microscopic domains of chemistry. Every measurement provides three kinds of information: the size or magnitude of the measurement (a number); a standard of comparison for the measurement (a unit); and an indication of the uncertainty of the measurement. While the number and unit are explicitly represented when a quantity is written, the uncertainty is an aspect of the measurement result that is more implicitly represented and will be discussed later.

The number in the measurement can be represented in different ways, including decimal form and scientific notation.

(Scientific notation is also known as exponential notation; a review of this topic can be found in Appendix B.) For example, the maximum takeoff weight of a Boeing 777-200ER airliner is 298,000 kilograms, which can also be written as $2.98 \times 10^{5} \mathrm{~kg}$. The mass of the average mosquito is about 0.0000025 kilograms, which can be written as $2.5 \times 10^{-6} \mathrm{~kg}$.

Units, such as liters, pounds, and centimeters, are standards of comparison for measurements. A 2-liter bottle of a soft drink contains a volume of beverage that is twice that of the accepted volume of 1 liter. The meat used to prepare a 0.25 -pound hamburger weighs one-fourth as much as the accepted weight of 1 pound. Without units, a number can be meaningless, confusing, or possibly life threatening. Suppose a doctor prescribes phenobarbital to control a patient's seizures and states a dosage of " 100 " without specifying units. Not only will this be confusing to the medical professional giving the dose, but the consequences can be dire: 100 mg given three times per day can be effective as an anticonvulsant, but a single dose of 100 g is more than 10 times the lethal amount.

The measurement units for seven fundamental properties ("base units") are listed in Table 1.2. The standards for these units are fixed by international agreement, and they are called the International System of Units or SI Units (from the French, Le Système International d'Unités). SI units have been used by the United States National Institute of Standards and Technology (NIST) since 1964. Units for other properties may be derived from these seven base units.

Base Units of the SI System

| Property Measured | Name of Unit | Symbol of Unit |
| :--: | :--: | :--: |
| length | meter | m |
| mass | kilogram | kg |
| time | second | s |
| temperature | kelvin | K |
| electric current | ampere | A |
| amount of substance | mole | mol |
| luminous intensity | candela | cd |

Table 1.2

Everyday measurement units are often defined as fractions or multiples of other units. Milk is commonly packaged in containers of 1 gallon ( 4 quarts), 1 quart ( 0.25 gallon), and one pint ( 0.5 quart). This same approach is used with SI units, but these fractions or multiples are always powers of 10. Fractional or multiple SI units are named using a prefix and the name of the base unit. For example, a length of 1000 meters is also called a kilometer because the prefix kilo means "one thousand," which in scientific notation is $10^{3}$ ( 1 kilometer $=1000 \mathrm{~m}=10^{3} \mathrm{~m}$ ). The prefixes used and the powers to which 10 are raised are listed in Table 1.3.

# Common Unit Prefixes 

| Prefix | Symbol | Factor | Example |
| :--: | :--: | :--: | :--: |
| femto | f | $10^{-15}$ | 1 femtosecond (fs) $=1 \times 10^{-15} \mathrm{~s}(0.00000000000001 \mathrm{~s})$ |
| pico | p | $10^{-12}$ | 1 picometer (pm) $=1 \times 10^{-12} \mathrm{~m}(0.000000000001 \mathrm{~m})$ |
| nano | n | $10^{-9}$ | 4 nanograms (ng) $=4 \times 10^{-9} \mathrm{~g}(0.000000004 \mathrm{~g})$ |
| micro | $\mu$ | $10^{-6}$ | 1 microliter $(\mu \mathrm{L})=1 \times 10^{-6} \mathrm{~L}(0.000001 \mathrm{~L})$ |

Table 1.3

Common Unit Prefixes

| Prefix | Symbol | Factor | Example |
| :--: | :--: | :--: | :--: |
| milli | m | $10^{-3}$ | 2 millimoles (mmol) $=2 \times 10^{-3} \mathrm{~mol}(0.002 \mathrm{~mol})$ |
| centi | c | $10^{-2}$ | 7 centimeters $(\mathrm{cm})=7 \times 10^{-2} \mathrm{~m}(0.07 \mathrm{~m})$ |
| deci | d | $10^{-1}$ | 1 deciliter $(\mathrm{dL})=1 \times 10^{-1} \mathrm{~L}(0.1 \mathrm{~L})$ |
| kilo | k | $10^{3}$ | 1 kilometer $(\mathrm{km})=1 \times 10^{3} \mathrm{~m}(1000 \mathrm{~m})$ |
| mega | M | $10^{6}$ | 3 megahertz $(\mathrm{MHz})=3 \times 10^{6} \mathrm{~Hz}(3,000,000 \mathrm{~Hz})$ |
| giga | G | $10^{9}$ | 8 gigayears $(\mathrm{Gyr})=8 \times 10^{9} \mathrm{yr}(8,000,000,000 \mathrm{yr})$ |
| tera | T | $10^{12}$ | 5 terawatts $(\mathrm{TW})=5 \times 10^{12} \mathrm{~W}(5,000,000,000,000 \mathrm{~W})$ |

Table 1.3

# Link to Learning 

Need a refresher or more practice with scientific notation? Visit this site (http://openstaxcollege.org/1/ 16notation) to go over the basics of scientific notation.

## SI Base Units

The initial units of the metric system, which eventually evolved into the SI system, were established in France during the French Revolution. The original standards for the meter and the kilogram were adopted there in 1799 and eventually by other countries. This section introduces four of the SI base units commonly used in chemistry. Other SI units will be introduced in subsequent chapters.

## Length

The standard unit of length in both the SI and original metric systems is the meter (m). A meter was originally specified as $1 / 10,000,000$ of the distance from the North Pole to the equator. It is now defined as the distance light in a vacuum travels in $1 / 299,792,458$ of a second. A meter is about 3 inches longer than a yard (Figure 1.23); one meter is about 39.37 inches or 1.094 yards. Longer distances are often reported in kilometers ( $1 \mathrm{~km}=1000 \mathrm{~m}=10^{3}$ m ), whereas shorter distances can be reported in centimeters $\left(1 \mathrm{~cm}=0.01 \mathrm{~m}=10^{-2} \mathrm{~m}\right)$ or millimeters $(1 \mathrm{~mm}=0.001$ $\mathrm{m}=10^{-3} \mathrm{~m}$ ).

![Image](Chapter_1_images/img-24.jpeg)

Figure 1.23 The relative lengths of $1 \mathrm{~m}, 1 \mathrm{yd}, 1 \mathrm{~cm}$, and 1 in . are shown (not actual size), as well as comparisons of 2.54 cm and 1 in ., and of 1 m and 1.094 yd .

# Mass 

The standard unit of mass in the SI system is the kilogram (kg). A kilogram was originally defined as the mass of a liter of water (a cube of water with an edge length of exactly 0.1 meter). It is now defined by a certain cylinder of platinum-iridium alloy, which is kept in France (Figure 1.24). Any object with the same mass as this cylinder is said to have a mass of 1 kilogram. One kilogram is about 2.2 pounds. The gram $(\mathrm{g})$ is exactly equal to $1 / 1000$ of the mass of the kilogram $\left(10^{-3} \mathrm{~kg}\right)$.
![Image](Chapter_1_images/img-25.jpeg)

Figure 1.24 This replica prototype kilogram is housed at the National Institute of Standards and Technology (NIST) in Maryland. (credit: National Institutes of Standards and Technology)

## Temperature

Temperature is an intensive property. The SI unit of temperature is the kelvin (K). The IUPAC convention is to use kelvin (all lowercase) for the word, K (uppercase) for the unit symbol, and neither the word "degree" nor the degree

symbol $\left({ }^{\circ}\right)$. The degree Celsius $\left({ }^{\circ} \mathbf{C}\right)$ is also allowed in the SI system, with both the word "degree" and the degree symbol used for Celsius measurements. Celsius degrees are the same magnitude as those of kelvin, but the two scales place their zeros in different places. Water freezes at $273.15 \mathrm{~K}\left(0^{\circ} \mathrm{C}\right)$ and boils at $373.15 \mathrm{~K}\left(100^{\circ} \mathrm{C}\right)$ by definition, and normal human body temperature is approximately $310 \mathrm{~K}\left(37^{\circ} \mathrm{C}\right)$. The conversion between these two units and the Fahrenheit scale will be discussed later in this chapter.

# Time 

The SI base unit of time is the second (s). Small and large time intervals can be expressed with the appropriate prefixes; for example, 3 microseconds $=0.000003 \mathrm{~s}=3 \times 10^{-6}$ and 5 megaseconds $=5,000,000 \mathrm{~s}=5 \times 10^{6} \mathrm{~s}$. Alternatively, hours, days, and years can be used.

## Derived SI Units

We can derive many units from the seven SI base units. For example, we can use the base unit of length to define a unit of volume, and the base units of mass and length to define a unit of density.

## Volume

Volume is the measure of the amount of space occupied by an object. The standard SI unit of volume is defined by the base unit of length (Figure 1.25). The standard volume is a cubic meter $\left(\mathbf{m}^{\mathbf{3}}\right)$, a cube with an edge length of exactly one meter. To dispense a cubic meter of water, we could build a cubic box with edge lengths of exactly one meter. This box would hold a cubic meter of water or any other substance.

A more commonly used unit of volume is derived from the decimeter ( 0.1 m , or 10 cm ). A cube with edge lengths of exactly one decimeter contains a volume of one cubic decimeter $\left(\mathrm{dm}^{3}\right)$. A liter ( $\mathbf{L}$ ) is the more common name for the cubic decimeter. One liter is about 1.06 quarts.

A cubic centimeter $\left(\mathbf{c m}^{\mathbf{3}}\right)$ is the volume of a cube with an edge length of exactly one centimeter. The abbreviation cc (for cubic centimeter) is often used by health professionals. A cubic centimeter is equivalent to a milliliter (mL) and is $1 / 1000$ of a liter.

![Image](Chapter_1_images/img-26.jpeg)

Figure 1.25 (a) The relative volumes are shown for cubes of $1 \mathrm{~m}^{3}, 1 \mathrm{dm}^{3}(1 \mathrm{~L})$, and $1 \mathrm{~cm}^{3}(1 \mathrm{~mL})$ (not to scale). (b) The diameter of a dime is compared relative to the edge length of a $1-\mathrm{cm}^{3}(1-\mathrm{mL})$ cube.

# Density 

We use the mass and volume of a substance to determine its density. Thus, the units of density are defined by the base units of mass and length.

The density of a substance is the ratio of the mass of a sample of the substance to its volume. The SI unit for density is the kilogram per cubic meter $\left(\mathrm{kg} / \mathrm{m}^{3}\right)$. For many situations, however, this as an inconvenient unit, and we often use grams per cubic centimeter $\left(\mathrm{g} / \mathrm{cm}^{3}\right)$ for the densities of solids and liquids, and grams per liter $(\mathrm{g} / \mathrm{L})$ for gases. Although there are exceptions, most liquids and solids have densities that range from about $0.7 \mathrm{~g} / \mathrm{cm}^{3}$ (the density of gasoline) to $19 \mathrm{~g} / \mathrm{cm}^{3}$ (the density of gold). The density of air is about $1.2 \mathrm{~g} / \mathrm{L}$. Table 1.4 shows the densities of some common substances.

Densities of Common Substances

| Solids | Liquids | Gases (at $25^{\circ} \mathrm{C}$ and 1 atm ) |
| :--: | :--: | :--: |
| ice (at $0^{\circ} \mathrm{C}$ ) $0.92 \mathrm{~g} / \mathrm{cm}^{3}$ | water $1.0 \mathrm{~g} / \mathrm{cm}^{3}$ | dry air $1.20 \mathrm{~g} / \mathrm{L}$ |
| oak (wood) $0.60-0.90 \mathrm{~g} / \mathrm{cm}^{3}$ | ethanol $0.79 \mathrm{~g} / \mathrm{cm}^{3}$ | oxygen $1.31 \mathrm{~g} / \mathrm{L}$ |
| iron $7.9 \mathrm{~g} / \mathrm{cm}^{3}$ | acetone $0.79 \mathrm{~g} / \mathrm{cm}^{3}$ | nitrogen $1.14 \mathrm{~g} / \mathrm{L}$ |
| copper $9.0 \mathrm{~g} / \mathrm{cm}^{3}$ | glycerin $1.26 \mathrm{~g} / \mathrm{cm}^{3}$ | carbon dioxide $1.80 \mathrm{~g} / \mathrm{L}$ |
| lead $11.3 \mathrm{~g} / \mathrm{cm}^{3}$ | olive oil $0.92 \mathrm{~g} / \mathrm{cm}^{3}$ | helium $0.16 \mathrm{~g} / \mathrm{L}$ |
| silver $10.5 \mathrm{~g} / \mathrm{cm}^{3}$ | gasoline $0.70-0.77 \mathrm{~g} / \mathrm{cm}^{3}$ | neon $0.83 \mathrm{~g} / \mathrm{L}$ |
| gold $19.3 \mathrm{~g} / \mathrm{cm}^{3}$ | mercury $13.6 \mathrm{~g} / \mathrm{cm}^{3}$ | radon $9.1 \mathrm{~g} / \mathrm{L}$ |

Table 1.4

While there are many ways to determine the density of an object, perhaps the most straightforward method involves

separately finding the mass and volume of the object, and then dividing the mass of the sample by its volume. In the following example, the mass is found directly by weighing, but the volume is found indirectly through length measurements.

$$
\text { density }=\frac{\text { mass }}{\text { volume }}
$$

# Example 1.1 

## Calculation of Density

Gold-in bricks, bars, and coins-has been a form of currency for centuries. In order to swindle people into paying for a brick of gold without actually investing in a brick of gold, people have considered filling the centers of hollow gold bricks with lead to fool buyers into thinking that the entire brick is gold. It does not work: Lead is a dense substance, but its density is not as great as that of gold, $19.3 \mathrm{~g} / \mathrm{cm}^{3}$. What is the density of lead if a cube of lead has an edge length of 2.00 cm and a mass of 90.7 g ?

## Solution

The density of a substance can be calculated by dividing its mass by its volume. The volume of a cube is calculated by cubing the edge length.

$$
\begin{gathered}
\text { volume of lead cube }=2.00 \mathrm{~cm} \times 2.00 \mathrm{~cm} \times 2.00 \mathrm{~cm}=8.00 \mathrm{~cm}^{3} \\
\text { density }=\frac{\text { mass }}{\text { volume }}=\frac{90.7 \mathrm{~g}}{8.00 \mathrm{~cm}^{3}}=\frac{11.3 \mathrm{~g}}{1.00 \mathrm{~cm}^{3}}=11.3 \mathrm{~g} / \mathrm{cm}^{3}
\end{gathered}
$$

(We will discuss the reason for rounding to the first decimal place in the next section.)

## Check Your Learning

(a) To three decimal places, what is the volume of a cube $\left(\mathrm{cm}^{3}\right)$ with an edge length of 0.843 cm ?
(b) If the cube in part (a) is copper and has a mass of 5.34 g , what is the density of copper to two decimal places?

Answer: (a) $0.599 \mathrm{~cm}^{3}$; (b) $8.91 \mathrm{~g} / \mathrm{cm}^{3}$

## Link to Learning

To learn more about the relationship between mass, volume, and density, use this interactive simulator (http://openstaxcollege.org///16phetmasvolden) to explore the density of different materials, like wood, ice, brick, and aluminum.

## Example 1.2

## Using Displacement of Water to Determine Density

This PhET simulation (http://openstaxcollege.org///16phetmasvolden) illustrates another way to determine density, using displacement of water. Determine the density of the red and yellow blocks.

## Solution

When you open the density simulation and select Same Mass, you can choose from several $5.00-\mathrm{kg}$ colored blocks that you can drop into a tank containing 100.00 L water. The yellow block floats (it is less dense than

water), and the water level rises to 105.00 L . While floating, the yellow block displaces 5.00 L water, an amount equal to the weight of the block. The red block sinks (it is more dense than water, which has density $=1.00 \mathrm{~kg} / \mathrm{L})$, and the water level rises to 101.25 L .

The red block therefore displaces 1.25 L water, an amount equal to the volume of the block. The density of the red block is:

$$
\text { density }=\frac{\text { mass }}{\text { volume }}=\frac{5.00 \mathrm{~kg}}{1.25 \mathrm{~L}}=4.00 \mathrm{~kg} / \mathrm{L}
$$

Note that since the yellow block is not completely submerged, you cannot determine its density from this information. But if you hold the yellow block on the bottom of the tank, the water level rises to 110.00 L , which means that it now displaces 10.00 L water, and its density can be found:

$$
\text { density }=\frac{\text { mass }}{\text { volume }}=\frac{5.00 \mathrm{~kg}}{10.00 \mathrm{~L}}=0.500 \mathrm{~kg} / \mathrm{L}
$$

# Check Your Learning 

Remove all of the blocks from the water and add the green block to the tank of water, placing it approximately in the middle of the tank. Determine the density of the green block.

Answer: $2.00 \mathrm{~kg} / \mathrm{L}$

### 1.5 Measurement Uncertainty, Accuracy, and Precision

By the end of this section, you will be able to:

- Define accuracy and precision
- Distinguish exact and uncertain numbers
- Correctly represent uncertainty in quantities using significant figures
- Apply proper rounding rules to computed quantities

Counting is the only type of measurement that is free from uncertainty, provided the number of objects being counted does not change while the counting process is underway. The result of such a counting measurement is an example of an exact number. By counting the eggs in a carton, one can determine exactly how many eggs the carton contains. The numbers of defined quantities are also exact. By definition, 1 foot is exactly 12 inches, 1 inch is exactly 2.54 centimeters, and 1 gram is exactly 0.001 kilogram. Quantities derived from measurements other than counting, however, are uncertain to varying extents due to practical limitations of the measurement process used.

## Significant Figures in Measurement

The numbers of measured quantities, unlike defined or directly counted quantities, are not exact. To measure the volume of liquid in a graduated cylinder, you should make a reading at the bottom of the meniscus, the lowest point on the curved surface of the liquid.

![Image](Chapter_1_images/img-27.jpeg)

Figure 1.26 To measure the volume of liquid in this graduated cylinder, you must mentally subdivide the distance between the 21 and 22 mL marks into tenths of a milliliter, and then make a reading (estimate) at the bottom of the meniscus.

Refer to the illustration in Figure 1.26. The bottom of the meniscus in this case clearly lies between the 21 and 22 markings, meaning the liquid volume is certainly greater than 21 mL but less than 22 mL . The meniscus appears to be a bit closer to the $22-\mathrm{mL}$ mark than to the $21-\mathrm{mL}$ mark, and so a reasonable estimate of the liquid's volume would be 21.6 mL . In the number 21.6, then, the digits 2 and 1 are certain, but the 6 is an estimate. Some people might estimate the meniscus position to be equally distant from each of the markings and estimate the tenth-place digit as 5 , while others may think it to be even closer to the $22-\mathrm{mL}$ mark and estimate this digit to be 7 . Note that it would be pointless to attempt to estimate a digit for the hundredths place, given that the tenths-place digit is uncertain. In general, numerical scales such as the one on this graduated cylinder will permit measurements to one-tenth of the smallest scale division. The scale in this case has $1-\mathrm{mL}$ divisions, and so volumes may be measured to the nearest 0.1 mL .

This concept holds true for all measurements, even if you do not actively make an estimate. If you place a quarter on a standard electronic balance, you may obtain a reading of 6.72 g . The digits 6 and 7 are certain, and the 2 indicates that the mass of the quarter is likely between 6.71 and 6.73 grams. The quarter weighs about 6.72 grams, with a nominal uncertainty in the measurement of $\pm 0.01$ gram. If the coin is weighed on a more sensitive balance, the mass might be 6.723 g . This means its mass lies between 6.722 and 6.724 grams, an uncertainty of 0.001 gram. Every measurement has some uncertainty, which depends on the device used (and the user's ability). All of the digits in a measurement, including the uncertain last digit, are called significant figures or significant digits. Note that zero may be a measured value; for example, if you stand on a scale that shows weight to the nearest pound and it shows "120," then the 1 (hundreds), 2 (tens) and 0 (ones) are all significant (measured) values.
A measurement result is properly reported when its significant digits accurately represent the certainty of the measurement process. But what if you were analyzing a reported value and trying to determine what is significant and what is not? Well, for starters, all nonzero digits are significant, and it is only zeros that require some thought. We will use the terms "leading," "trailing," and "captive" for the zeros and will consider how to deal with them.

![Image](Chapter_1_images/img-28.jpeg)

Starting with the first nonzero digit on the left, count this digit and all remaining digits to the right. This is the number of significant figures in the measurement unless the last digit is a trailing zero lying to the left of the decimal point.
![Image](Chapter_1_images/img-29.jpeg)

Captive zeros result from measurement and are therefore always significant. Leading zeros, however, are never significant-they merely tell us where the decimal point is located.
![Image](Chapter_1_images/img-30.jpeg)

The leading zeros in this example are not significant. We could use exponential notation (as described in Appendix B) and express the number as $8.32407 \times 10^{-3}$; then the number 8.32407 contains all of the significant figures, and $10^{-3}$ locates the decimal point.

The number of significant figures is uncertain in a number that ends with a zero to the left of the decimal point location. The zeros in the measurement 1,300 grams could be significant or they could simply indicate where the decimal point is located. The ambiguity can be resolved with the use of exponential notation: $1.3 \times 10^{3}$ (two significant figures), $1.30 \times 10^{3}$ (three significant figures, if the tens place was measured), or $1.300 \times 10^{3}$ (four significant figures, if the ones place was also measured). In cases where only the decimal-formatted number is available, it is prudent to assume that all trailing zeros are not significant.
![Image](Chapter_1_images/img-31.jpeg)

When determining significant figures, be sure to pay attention to reported values and think about the measurement and significant figures in terms of what is reasonable or likely when evaluating whether the value makes sense. For example, the official January 2014 census reported the resident population of the US as $317,297,725$. Do you think the US population was correctly determined to the reported nine significant figures, that is, to the exact number of people? People are constantly being born, dying, or moving into or out of the country, and assumptions are made to account for the large number of people who are not actually counted. Because of these uncertainties, it might be more reasonable to expect that we know the population to within perhaps a million or so, in which case the population should be reported as $3.17 \times 10^{8}$ people.

# Significant Figures in Calculations 

A second important principle of uncertainty is that results calculated from a measurement are at least as uncertain as

the measurement itself. Take the uncertainty in measurements into account to avoid misrepresenting the uncertainty in calculated results. One way to do this is to report the result of a calculation with the correct number of significant figures, which is determined by the following three rules for rounding numbers:

1. When adding or subtracting numbers, round the result to the same number of decimal places as the number with the least number of decimal places (the least certain value in terms of addition and subtraction).
2. When multiplying or dividing numbers, round the result to the same number of digits as the number with the least number of significant figures (the least certain value in terms of multiplication and division).
3. If the digit to be dropped (the one immediately to the right of the digit to be retained) is less than 5, "round down" and leave the retained digit unchanged; if it is more than 5, "round up" and increase the retained digit by 1 ; if the dropped digit is 5 , round up or down, whichever yields an even value for the retained digit. (The last part of this rule may strike you as a bit odd, but it's based on reliable statistics and is aimed at avoiding any bias when dropping the digit " 5 ," since it is equally close to both possible values of the retained digit.)

The following examples illustrate the application of this rule in rounding a few different numbers to three significant figures:

- 0.028675 rounds "up" to 0.0287 (the dropped digit, 7, is greater than 5)
- 18.3384 rounds "down" to 18.3 (the dropped digit, 3, is less than 5)
- 6.8752 rounds "up" to 6.88 (the dropped digit is 5 , and the retained digit is even)
- 92.85 rounds "down" to 92.8 (the dropped digit is 5 , and the retained digit is even)

Let's work through these rules with a few examples.

# Example 1.3 

## Rounding Numbers

Round the following to the indicated number of significant figures:
(a) 31.57 (to two significant figures)
(b) 8.1649 (to three significant figures)
(c) 0.051065 (to four significant figures)
(d) 0.90275 (to four significant figures)

## Solution

(a) 31.57 rounds "up" to 32 (the dropped digit is 5 , and the retained digit is even)
(b) 8.1649 rounds "down" to 8.16 (the dropped digit, 4 , is less than 5 )
(c) 0.051065 rounds "down" to 0.05106 (the dropped digit is 5 , and the retained digit is even)
(d) 0.90275 rounds "up" to 0.9028 (the dropped digit is 5 , and the retained digit is even)

## Check Your Learning

Round the following to the indicated number of significant figures:
(a) 0.424 (to two significant figures)
(b) 0.0038661 (to three significant figures)
(c) 421.25 (to four significant figures)
(d) 28,683.5 (to five significant figures)

Answer: (a) 0.42 ; (b) 0.00387 ; (c) 421.2 ; (d) 28,684

## Example 1.4

## Addition and Subtraction with Significant Figures

Rule: When adding or subtracting numbers, round the result to the same number of decimal places as the number with the fewest decimal places (i.e., the least certain value in terms of addition and subtraction).
(a) Add 1.0023 g and 4.383 g .
(b) Subtract 421.23 g from 486 g .

## Solution

(a) $\frac{1.0023 \mathrm{~g}}{+4.383 \mathrm{~g}}$

Answer is 5.385 g (round to the thousandths place; three decimal places)
(b) $\frac{486 \mathrm{~g}}{-421.23 \mathrm{~g}}$

Answer is 65 g (round to the ones place; no decimal places)

![Image](Chapter_1_images/img-32.jpeg)

# Check Your Learning 

(a) Add 2.334 mL and 0.31 mL .
(b) Subtract 55.8752 m from 56.533 m .

Answer: (a) 2.64 mL ; (b) 0.658 m

## Example 1.5

## Multiplication and Division with Significant Figures

Rule: When multiplying or dividing numbers, round the result to the same number of digits as the number with the fewest significant figures (the least certain value in terms of multiplication and division).
(a) Multiply 0.6238 cm by 6.6 cm .
(b) Divide 421.23 g by 486 mL .

## Solution

(a) $0.6238 \mathrm{~cm} \times 6.6 \mathrm{~cm}=4.11708 \mathrm{~cm}^{2} \longrightarrow$ result is $4.1 \mathrm{~cm}^{2}$ (round to two significant fig es)
four significant fig es $\times$ two significant fig es $\longrightarrow$ two significant fig es answer
$\frac{421.23 \mathrm{~g}}{486 \mathrm{~mL}}=0.86728 \ldots \mathrm{~g} / \mathrm{mL} \longrightarrow$ result is $0.867 \mathrm{~g} / \mathrm{mL}$ (round to three significant fig es)
(b) $\frac{\text { fi e significant fig es }}{\text { three significant fig es }} \longrightarrow$ three significant fig es answer

## Check Your Learning

(a) Multiply 2.334 cm and 0.320 cm .
(b) Divide 55.8752 m by 56.53 s .

Answer: (a) $0.747 \mathrm{~cm}^{2}$ (b) $0.9884 \mathrm{~m} / \mathrm{s}$
In the midst of all these technicalities, it is important to keep in mind the reason for these rules about significant figures and rounding - to correctly represent the certainty of the values reported and to ensure that a calculated result is not represented as being more certain than the least certain value used in the calculation.

## Example 1.6

## Calculation with Significant Figures

One common bathtub is 13.44 dm long, 5.920 dm wide, and 2.54 dm deep. Assume that the tub is rectangular and calculate its approximate volume in liters.

## Solution

$$
\begin{aligned}
V & =l \times w \times d \\
& =13.44 \mathrm{dm} \times 5.920 \mathrm{dm} \times 2.54 \mathrm{dm} \\
& =202.09459 \ldots \mathrm{dm}^{3}(\text { value from calculator }) \\
& =202 \mathrm{dm}^{3}, \text { or } 202 \mathrm{~L}(\text { answer rounded to three significant fig es) }
\end{aligned}
$$

# Check Your Learning 

What is the density of a liquid with a mass of 31.1415 g and a volume of $30.13 \mathrm{~cm}^{3}$ ?
Answer: $1.034 \mathrm{~g} / \mathrm{mL}$

## Example 1.7

## Experimental Determination of Density Using Water Displacement

A piece of rebar is weighed and then submerged in a graduated cylinder partially filled with water, with results as shown.
![Image](Chapter_1_images/img-33.jpeg)
(a) Use these values to determine the density of this piece of rebar.
(b) Rebar is mostly iron. Does your result in (a) support this statement? How?

## Solution

The volume of the piece of rebar is equal to the volume of the water displaced:

$$
\text { volume }=22.4 \mathrm{~mL}-13.5 \mathrm{~mL}=8.9 \mathrm{~mL}=8.9 \mathrm{~cm}^{3}
$$

(rounded to the nearest 0.1 mL , per the rule for addition and subtraction)
The density is the mass-to-volume ratio:

$$
\text { density }=\frac{\text { mass }}{\text { volume }}=\frac{69.658 \mathrm{~g}}{8.9 \mathrm{~cm}^{3}}=7.8 \mathrm{~g} / \mathrm{cm}^{3}
$$

(rounded to two significant figures, per the rule for multiplication and division)

From Table 1.4, the density of iron is $7.9 \mathrm{~g} / \mathrm{cm}^{3}$, very close to that of rebar, which lends some support to the fact that rebar is mostly iron.

# Check Your Learning 

An irregularly shaped piece of a shiny yellowish material is weighed and then submerged in a graduated cylinder, with results as shown.
![Image](Chapter_1_images/img-34.jpeg)
(a) Use these values to determine the density of this material.
(b) Do you have any reasonable guesses as to the identity of this material? Explain your reasoning.

Answer: (a) $19 \mathrm{~g} / \mathrm{cm}^{3}$; (b) It is likely gold; the right appearance for gold and very close to the density given for gold in Table 1.4.

## Accuracy and Precision

Scientists typically make repeated measurements of a quantity to ensure the quality of their findings and to evaluate both the precision and the accuracy of their results. Measurements are said to be precise if they yield very similar results when repeated in the same manner. A measurement is considered accurate if it yields a result that is very close to the true or accepted value. Precise values agree with each other; accurate values agree with a true value. These characterizations can be extended to other contexts, such as the results of an archery competition (Figure 1.27).

![Image](Chapter_1_images/img-35.jpeg)

Figure 1.27 (a) These arrows are close to both the bull's eye and one another, so they are both accurate and precise. (b) These arrows are close to one another but not on target, so they are precise but not accurate. (c) These arrows are neither on target nor close to one another, so they are neither accurate nor precise.

Suppose a quality control chemist at a pharmaceutical company is tasked with checking the accuracy and precision of three different machines that are meant to dispense 10 ounces ( 296 mL ) of cough syrup into storage bottles. She proceeds to use each machine to fill five bottles and then carefully determines the actual volume dispensed, obtaining the results tabulated in Table 1.5.

Volume (mL) of Cough Medicine Delivered by 10-oz (296 mL) Dispensers

| Dispenser \#1 | Dispenser \#2 | Dispenser \#3 |
| :--: | :--: | :--: |
| 283.3 | 298.3 | 296.1 |
| 284.1 | 294.2 | 295.9 |
| 283.9 | 296.0 | 296.1 |
| 284.0 | 297.8 | 296.0 |
| 284.1 | 293.9 | 296.1 |

Table 1.5

Considering these results, she will report that dispenser \#1 is precise (values all close to one another, within a few tenths of a milliliter) but not accurate (none of the values are close to the target value of 296 mL , each being more than 10 mL too low). Results for dispenser \#2 represent improved accuracy (each volume is less than 3 mL away from 296 mL ) but worse precision (volumes vary by more than 4 mL ). Finally, she can report that dispenser \#3 is working well, dispensing cough syrup both accurately (all volumes within 0.1 mL of the target volume) and precisely (volumes differing from each other by no more than 0.2 mL ).

# 1.6 Mathematical Treatment of Measurement Results 

By the end of this section, you will be able to:

- Explain the dimensional analysis (factor label) approach to mathematical calculations involving quantities
- Use dimensional analysis to carry out unit conversions for a given property and computations involving two or more properties

It is often the case that a quantity of interest may not be easy (or even possible) to measure directly but instead must be calculated from other directly measured properties and appropriate mathematical relationships. For example, consider measuring the average speed of an athlete running sprints. This is typically accomplished by measuring the time required for the athlete to run from the starting line to the finish line, and the distance between these two lines, and then computing speed from the equation that relates these three properties:

$$
\text { speed }=\frac{\text { distance }}{\text { time }}
$$

An Olympic-quality sprinter can run 100 m in approximately 10 s , corresponding to an average speed of

$$
\frac{100 \mathrm{~m}}{10 \mathrm{~s}}=10 \mathrm{~m} / \mathrm{s}
$$

Note that this simple arithmetic involves dividing the numbers of each measured quantity to yield the number of the computed quantity $(100 / 10=10)$ and likewise dividing the units of each measured quantity to yield the unit of the computed quantity $(\mathrm{m} / \mathrm{s}=\mathrm{m} / \mathrm{s})$. Now, consider using this same relation to predict the time required for a person running at this speed to travel a distance of 25 m . The same relation among the three properties is used, but in this case, the two quantities provided are a speed $(10 \mathrm{~m} / \mathrm{s})$ and a distance $(25 \mathrm{~m})$. To yield the sought property, time, the equation must be rearranged appropriately:

$$
\text { time }=\frac{\text { distance }}{\text { speed }}
$$

The time can then be computed as:

$$
\frac{25 \mathrm{~m}}{10 \mathrm{~m} / \mathrm{s}}=2.5 \mathrm{~s}
$$

Again, arithmetic on the numbers $(25 / 10=2.5)$ was accompanied by the same arithmetic on the units $(\mathrm{m} / \mathrm{m} / \mathrm{s}=\mathrm{s})$ to yield the number and unit of the result, 2.5 s . Note that, just as for numbers, when a unit is divided by an identical unit (in this case, $\mathrm{m} / \mathrm{m}$ ), the result is " 1 "-or, as commonly phrased, the units "cancel."

These calculations are examples of a versatile mathematical approach known as dimensional analysis (or the factorlabel method). Dimensional analysis is based on this premise: the units of quantities must be subjected to the same mathematical operations as their associated numbers. This method can be applied to computations ranging from simple unit conversions to more complex, multi-step calculations involving several different quantities.

# Conversion Factors and Dimensional Analysis 

A ratio of two equivalent quantities expressed with different measurement units can be used as a unit conversion factor. For example, the lengths of 2.54 cm and 1 in . are equivalent (by definition), and so a unit conversion factor may be derived from the ratio,

$$
\frac{2.54 \mathrm{~cm}}{1 \mathrm{in} .}(2.54 \mathrm{~cm}=1 \mathrm{in} .) \text { or } 2.54 \frac{\mathrm{~cm}}{\mathrm{in} .}
$$

Several other commonly used conversion factors are given in Table 1.6.
Common Conversion Factors

| Length | Volume | Mass |
| :--: | :--: | :--: |
| $1 \mathrm{~m}=1.0936 \mathrm{yd}$ | $1 \mathrm{~L}=1.0567 \mathrm{qt}$ | $1 \mathrm{~kg}=2.2046 \mathrm{lb}$ |
| $1 \mathrm{in} .=2.54 \mathrm{~cm}$ (exact) | $1 \mathrm{qt}=0.94635 \mathrm{~L}$ | $1 \mathrm{lb}=453.59 \mathrm{~g}$ |
| $1 \mathrm{~km}=0.62137 \mathrm{mi}$ | $1 \mathrm{ft}^{3}=28.317 \mathrm{~L}$ | 1 (avoirdupois) oz $=28.349 \mathrm{~g}$ |
| $1 \mathrm{mi}=1609.3 \mathrm{~m}$ | $1 \mathrm{tbsp}=14.787 \mathrm{~mL}$ | 1 (troy) oz $=31.103 \mathrm{~g}$ |

Table 1.6

When a quantity (such as distance in inches) is multiplied by an appropriate unit conversion factor, the quantity is converted to an equivalent value with different units (such as distance in centimeters). For example, a basketball player's vertical jump of 34 inches can be converted to centimeters by:

$$
34 \text { in. } \times \frac{2.54 \mathrm{~cm}}{1 \mathrm{in}}=86 \mathrm{~cm}
$$

Since this simple arithmetic involves quantities, the premise of dimensional analysis requires that we multiply both numbers and units. The numbers of these two quantities are multiplied to yield the number of the product quantity, 86, whereas the units are multiplied to yield $\frac{\mathrm{in} . \times \mathrm{cm}}{\mathrm{in} .}$. Just as for numbers, a ratio of identical units is also numerically equal to one, $\frac{\mathrm{in} .}{\mathrm{in} .}=1$, and the unit product thus simplifies to $c m$. (When identical units divide to yield a factor of 1, they are said to "cancel.") Dimensional analysis may be used to confirm the proper application of unit conversion factors as demonstrated in the following example.

# Example 1.8 

## Using a Unit Conversion Factor

The mass of a competition frisbee is 125 g . Convert its mass to ounces using the unit conversion factor derived from the relationship $1 \mathrm{oz}=28.349 \mathrm{~g}$ (Table 1.6).

## Solution

Given the conversion factor, the mass in ounces may be derived using an equation similar to the one used for converting length from inches to centimeters.

$$
x \mathrm{oz}=125 \mathrm{~g} \times \text { unit conversion factor }
$$

The unit conversion factor may be represented as:

$$
\frac{1 \mathrm{oz}}{28.349 \mathrm{~g}} \text { and } \frac{28.349 \mathrm{~g}}{1 \mathrm{oz}}
$$

The correct unit conversion factor is the ratio that cancels the units of grams and leaves ounces.

$$
\begin{aligned}
x \mathrm{oz} & =125 \# \times \frac{1 \mathrm{oz}}{28.349 \#} \\
& =\left(\frac{125}{28.349}\right) \mathrm{oz} \\
& =4.41 \mathrm{oz}(\text { three significant fig es) }
\end{aligned}
$$

## Check Your Learning

Convert a volume of 9.345 qt to liters.

Beyond simple unit conversions, the factor-label method can be used to solve more complex problems involving computations. Regardless of the details, the basic approach is the same-all the factors involved in the calculation must be appropriately oriented to ensure that their labels (units) will appropriately cancel and/or combine to yield the desired unit in the result. As your study of chemistry continues, you will encounter many opportunities to apply this approach.

# Example 1.9 

## Computing Quantities from Measurement Results and Known Mathematical Relations

What is the density of common antifreeze in units of $\mathrm{g} / \mathrm{mL}$ ? A $4.00-\mathrm{qt}$ sample of the antifreeze weighs 9.26 lb .

## Solution

Since density $=\frac{\text { mass }}{\text { volume }}$, we need to divide the mass in grams by the volume in milliliters. In general: the number of units of $B=$ the number of units of $A \times$ unit conversion factor. The necessary conversion factors are given in Table 1.6: $1 \mathrm{lb}=453.59 \mathrm{~g} ; 1 \mathrm{~L}=1.0567 \mathrm{qt} ; 1 \mathrm{~L}=1,000 \mathrm{~mL}$. Mass may be converted from pounds to grams as follows:

$$
9.26 \# \times \frac{453.59 \mathrm{~g}}{1 \#}=4.20 \times 10^{3} \mathrm{~g}
$$

Volume may be converted from quarts to millimeters via two steps:
Step 1. Convert quarts to liters.

$$
4.00 \text { qt } \times \frac{1 \mathrm{~L}}{1.0567 \text { qt }}=3.78 \mathrm{~L}
$$

Step 2. Convert liters to milliliters.

$$
3.78 \pm \times \frac{1000 \mathrm{~mL}}{1 \mathrm{t}}=3.78 \times 10^{3} \mathrm{~mL}
$$

Then,

$$
\text { density }=\frac{4.20 \times 10^{3} \mathrm{~g}}{3.78 \times 10^{3} \mathrm{~mL}}=1.11 \mathrm{~g} / \mathrm{mL}
$$

Alternatively, the calculation could be set up in a way that uses three unit conversion factors sequentially as follows:

$$
\frac{9.26 \#}{4.00 \text { qt }} \times \frac{453.59 \mathrm{~g}}{1 \# \#} \times \frac{1.0567 \text { qt }}{1 \mathrm{t}} \times \frac{1 \mathrm{t}}{1000 \mathrm{~mL}}=1.11 \mathrm{~g} / \mathrm{mL}
$$

## Check Your Learning

What is the volume in liters of 1.000 oz , given that $1 \mathrm{~L}=1.0567 \mathrm{qt}$ and $1 \mathrm{qt}=32 \mathrm{oz}$ (exactly)?
Answer: $2.956 \times 10^{-2} \mathrm{~L}$

## Example 1.10

## Computing Quantities from Measurement Results and Known Mathematical Relations

While being driven from Philadelphia to Atlanta, a distance of about 1250 km, a 2014 Lamborghini Aventador Roadster uses 213 L gasoline.
(a) What (average) fuel economy, in miles per gallon, did the Roadster get during this trip?

(b) If gasoline costs $\$ 3.80$ per gallon, what was the fuel cost for this trip?

# Solution 

(a) First convert distance from kilometers to miles:

$$
1250 \mathrm{~km} \times \frac{0.62137 \mathrm{mi}}{1 \mathrm{~km}}=777 \mathrm{mi}
$$

and then convert volume from liters to gallons:

$$
213 \mathrm{~L} \times \frac{1.0567 \mathrm{mi}}{1 \mathrm{~L}} \times \frac{1 \mathrm{gal}}{4 \mathrm{mi}}=56.3 \mathrm{gal}
$$

Finally,

$$
\text { (average) mileage }=\frac{777 \mathrm{mi}}{56.3 \mathrm{gal}}=13.8 \text { miles } / \text { gallon }=13.8 \mathrm{mpg}
$$

Alternatively, the calculation could be set up in a way that uses all the conversion factors sequentially, as follows:

$$
\frac{1250 \mathrm{~km}}{213 \mathrm{~L}} \times \frac{0.62137 \mathrm{mi}}{1 \mathrm{~km}} \times \frac{1 \mathrm{~L}}{1.0567 \mathrm{mi}} \times \frac{4 \mathrm{mi}}{1 \mathrm{gal}}=13.8 \mathrm{mpg}
$$

(b) Using the previously calculated volume in gallons, we find:

$$
56.3 \mathrm{gal} \times \frac{\$ 3.80}{1 \mathrm{gal}}=\$ 214
$$

## Check Your Learning

A Toyota Prius Hybrid uses 59.7 L gasoline to drive from San Francisco to Seattle, a distance of 1300 km (two significant digits).
(a) What (average) fuel economy, in miles per gallon, did the Prius get during this trip?
(b) If gasoline costs $\$ 3.90$ per gallon, what was the fuel cost for this trip?

Answer: (a) 51 mpg ; (b) $\$ 62$

## Conversion of Temperature Units

We use the word temperature to refer to the hotness or coldness of a substance. One way we measure a change in temperature is to use the fact that most substances expand when their temperature increases and contract when their temperature decreases. The mercury or alcohol in a common glass thermometer changes its volume as the temperature changes, and the position of the trapped liquid along a printed scale may be used as a measure of temperature.

Temperature scales are defined relative to selected reference temperatures: Two of the most commonly used are the freezing and boiling temperatures of water at a specified atmospheric pressure. On the Celsius scale, $0^{\circ} \mathrm{C}$ is defined as the freezing temperature of water and $100^{\circ} \mathrm{C}$ as the boiling temperature of water. The space between the two temperatures is divided into 100 equal intervals, which we call degrees. On the Fahrenheit scale, the freezing point of water is defined as $32^{\circ} \mathrm{F}$ and the boiling temperature as $212^{\circ} \mathrm{F}$. The space between these two points on a Fahrenheit thermometer is divided into 180 equal parts (degrees).

Defining the Celsius and Fahrenheit temperature scales as described in the previous paragraph results in a slightly more complex relationship between temperature values on these two scales than for different units of measure for other properties. Most measurement units for a given property are directly proportional to one another ( $y=m x$ ). Using familiar length units as one example:

$$
\text { length in feet }=\left(\frac{1 \mathrm{ft}}{12 \mathrm{in}}\right) \times \text { length in inches }
$$

where $\mathrm{y}=$ length in feet, $\mathrm{x}=$ length in inches, and the proportionality constant, m , is the conversion factor. The Celsius

and Fahrenheit temperature scales, however, do not share a common zero point, and so the relationship between these two scales is a linear one rather than a proportional one $(\mathrm{y}=\mathrm{mx}+\mathrm{b})$. Consequently, converting a temperature from one of these scales into the other requires more than simple multiplication by a conversion factor, $m$, it also must take into account differences in the scales' zero points (b).

The linear equation relating Celsius and Fahrenheit temperatures is easily derived from the two temperatures used to define each scale. Representing the Celsius temperature as $x$ and the Fahrenheit temperature as $y$, the slope, $m$, is computed to be:

$$
m=\frac{\Delta y}{\Delta x}=\frac{212}{100}^{\circ} \mathrm{F}-\frac{32}{100}^{\circ} \mathrm{F}=\frac{180}{100}^{\circ} \mathrm{F}=\frac{9}{5}^{\circ} \mathrm{F}
$$

The $y$-intercept of the equation, $b$, is then calculated using either of the equivalent temperature pairs, $\left(100^{\circ} \mathrm{C}, 212^{\circ} \mathrm{F}\right)$ or $\left(0^{\circ} \mathrm{C}, 32^{\circ} \mathrm{F}\right)$, as:

$$
b=y-m x=32^{\circ} \mathrm{F}-\frac{9}{5}^{\circ} \mathrm{F} \times 0^{\circ} \mathrm{C}=32^{\circ} \mathrm{F}
$$

The equation relating the temperature $(T)$ scales is then:

$$
T_{\circ \mathrm{F}}=\left(\frac{9}{5}^{\circ} \mathrm{F} \times T_{\circ} \mathrm{C}\right)+32^{\circ} \mathrm{C}
$$

An abbreviated form of this equation that omits the measurement units is:

$$
T_{\circ \mathrm{F}}=\left(\frac{9}{5} \times T_{\circ} \mathrm{C}\right)+32
$$

Rearrangement of this equation yields the form useful for converting from Fahrenheit to Celsius:

$$
T_{\circ} \mathrm{C}=\frac{5}{9}\left(T_{\circ} \mathrm{F}-32\right)
$$

As mentioned earlier in this chapter, the SI unit of temperature is the kelvin (K). Unlike the Celsius and Fahrenheit scales, the kelvin scale is an absolute temperature scale in which 0 (zero) K corresponds to the lowest temperature that can theoretically be achieved. Since the kelvin temperature scale is absolute, a degree symbol is not included in the unit abbreviation, K. The early 19th-century discovery of the relationship between a gas's volume and temperature suggested that the volume of a gas would be zero at $-273.15^{\circ} \mathrm{C}$. In 1848, British physicist William Thompson, who later adopted the title of Lord Kelvin, proposed an absolute temperature scale based on this concept (further treatment of this topic is provided in this text's chapter on gases).

The freezing temperature of water on this scale is 273.15 K and its boiling temperature is 373.15 K . Notice the numerical difference in these two reference temperatures is 100 , the same as for the Celsius scale, and so the linear relation between these two temperature scales will exhibit a slope of $1 \frac{\mathrm{~K}}{\circ} \mathrm{C}$. Following the same approach, the equations for converting between the kelvin and Celsius temperature scales are derived to be:

$$
\begin{aligned}
& T_{\mathrm{K}}=T_{\circ \mathrm{C}}+273.15 \\
& T_{\circ} \mathrm{C}=T_{\mathrm{K}}-273.15
\end{aligned}
$$

The 273.15 in these equations has been determined experimentally, so it is not exact. Figure 1.28 shows the relationship among the three temperature scales.

![Image](Chapter_1_images/img-36.jpeg)

Figure 1.28 The Fahrenheit, Celsius, and kelvin temperature scales are compared.

Although the kelvin (absolute) temperature scale is the official SI temperature scale, Celsius is commonly used in many scientific contexts and is the scale of choice for nonscience contexts in almost all areas of the world. Very few countries (the U.S. and its territories, the Bahamas, Belize, Cayman Islands, and Palau) still use Fahrenheit for weather, medicine, and cooking.

# Example 1.11 

## Conversion from Celsius

Normal body temperature has been commonly accepted as $37.0^{\circ} \mathrm{C}$ (although it varies depending on time of day and method of measurement, as well as among individuals). What is this temperature on the kelvin scale and on the Fahrenheit scale?

## Solution

$$
\begin{gathered}
\mathrm{K}={ }^{\circ} \mathrm{C}+273.15=37.0+273.2=310.2 \mathrm{~K} \\
{ }^{\circ} \mathrm{F}=\frac{9}{5}{ }^{\circ} \mathrm{C}+32.0=\left(\frac{9}{5} \times 37.0\right)+32.0=66.6+32.0=98.6^{\circ} \mathrm{F}
\end{gathered}
$$

## Check Your Learning

Convert $80.92^{\circ} \mathrm{C}$ to K and ${ }^{\circ} \mathrm{F}$.
Answer: $354.07 \mathrm{~K}, 177.7^{\circ} \mathrm{F}$

## Example 1.12

## Conversion from Fahrenheit

Baking a ready-made pizza calls for an oven temperature of $450^{\circ} \mathrm{F}$. If you are in Europe, and your oven thermometer uses the Celsius scale, what is the setting? What is the kelvin temperature?

## Solution

$$
\begin{gathered}
{ }^{\circ} \mathrm{C}=\frac{5}{9}\left({ }^{\circ} \mathrm{F}-32\right)=\frac{5}{9}(450-32)=\frac{5}{9} \times 418=232^{\circ} \mathrm{C} \longrightarrow \text { set oven to } 230^{\circ} \mathrm{C} \quad \text { (two significant fig es) } \\
\mathrm{K}={ }^{\circ} \mathrm{C}+273.15=230+273=503 \mathrm{~K} \longrightarrow 5.0 \times 10^{2} \mathrm{~K} \quad \text { (two significant fig es) }
\end{gathered}
$$

## Check Your Learning

Convert $50^{\circ} \mathrm{F}$ to ${ }^{\circ} \mathrm{C}$ and K .
Answer: $10^{\circ} \mathrm{C}, 280 \mathrm{~K}$

# Key Terms 

accuracy how closely a measurement aligns with a correct value
atom smallest particle of an element that can enter into a chemical combination
Celsius $\left({ }^{\circ} \mathbf{C}\right)$ unit of temperature; water freezes at $0^{\circ} \mathrm{C}$ and boils at $100^{\circ} \mathrm{C}$ on this scale
chemical change change producing a different kind of matter from the original kind of matter
chemical property behavior that is related to the change of one kind of matter into another kind of matter
chemistry study of the composition, properties, and interactions of matter
compound pure substance that can be decomposed into two or more elements
cubic centimeter $\left(\mathbf{c m}^{3}\right.$ or cc ) volume of a cube with an edge length of exactly 1 cm
cubic meter $\left(\mathbf{m}^{3}\right)$ SI unit of volume
density ratio of mass to volume for a substance or object
dimensional analysis (also, factor-label method) versatile mathematical approach that can be applied to computations ranging from simple unit conversions to more complex, multi-step calculations involving several different quantities
element substance that is composed of a single type of atom; a substance that cannot be decomposed by a chemical change
exact number number derived by counting or by definition
extensive property property of a substance that depends on the amount of the substance
Fahrenheit unit of temperature; water freezes at $32^{\circ} \mathrm{F}$ and boils at $212^{\circ} \mathrm{F}$ on this scale
gas state in which matter has neither definite volume nor shape
heterogeneous mixture combination of substances with a composition that varies from point to point
homogeneous mixture (also, solution) combination of substances with a composition that is uniform throughout
hypothesis tentative explanation of observations that acts as a guide for gathering and checking information
intensive property property of a substance that is independent of the amount of the substance
kelvin (K) SI unit of temperature; $273.15 \mathrm{~K}=0^{\circ} \mathrm{C}$
kilogram (kg) standard SI unit of mass; 1 kg = approximately 2.2 pounds
law statement that summarizes a vast number of experimental observations, and describes or predicts some aspect of the natural world
law of conservation of matter when matter converts from one type to another or changes form, there is no detectable change in the total amount of matter present
length measure of one dimension of an object
liquid state of matter that has a definite volume but indefinite shape

liter (L) (also, cubic decimeter) unit of volume; $1 \mathrm{~L}=1,000 \mathrm{~cm}^{3}$
macroscopic domain realm of everyday things that are large enough to sense directly by human sight and touch
mass fundamental property indicating amount of matter
matter anything that occupies space and has mass
meter (m) standard metric and SI unit of length; $1 \mathrm{~m}=$ approximately 1.094 yards
microscopic domain realm of things that are much too small to be sensed directly
milliliter (mL) 1/1,000 of a liter; equal to $1 \mathrm{~cm}^{3}$
mixture matter that can be separated into its components by physical means
molecule bonded collection of two or more atoms of the same or different elements
physical change change in the state or properties of matter that does not involve a change in its chemical composition
physical property characteristic of matter that is not associated with any change in its chemical composition
plasma gaseous state of matter containing a large number of electrically charged atoms and/or molecules
precision how closely a measurement matches the same measurement when repeated
pure substance homogeneous substance that has a constant composition
rounding procedure used to ensure that calculated results properly reflect the uncertainty in the measurements used in the calculation
scientific method path of discovery that leads from question and observation to law or hypothesis to theory, combined with experimental verification of the hypothesis and any necessary modification of the theory
second (s) SI unit of time
SI units (International System of Units) standards fixed by international agreement in the International System of Units (Le Système International d'Unités)
significant figures (also, significant digits) all of the measured digits in a determination, including the uncertain last digit
solid state of matter that is rigid, has a definite shape, and has a fairly constant volume
symbolic domain specialized language used to represent components of the macroscopic and microscopic domains, such as chemical symbols, chemical formulas, chemical equations, graphs, drawings, and calculations
temperature intensive property representing the hotness or coldness of matter
theory well-substantiated, comprehensive, testable explanation of a particular aspect of nature
uncertainty estimate of amount by which measurement differs from true value
unit standard of comparison for measurements
unit conversion factor ratio of equivalent quantities expressed with different units; used to convert from one unit to a different unit

volume amount of space occupied by an object
weight force that gravity exerts on an object

# Key Equations 

- density $=\frac{\text { mass }}{\text { volume }}$
- $T_{\circ C}=\frac{5}{9} \times\left(T_{\circ F}-32\right)$
- $T_{\circ F}=\left(\frac{9}{5} \times T_{\circ}\right)+32$
- $T_{\mathrm{K}}={ }^{\circ} \mathrm{C}+273.15$
- $T_{\circ \mathrm{C}}=\mathrm{K}-273.15$


## Summary

### 1.1 Chemistry in Context

Chemistry deals with the composition, structure, and properties of matter, and the ways by which various forms of matter may be interconverted. Thus, it occupies a central place in the study and practice of science and technology. Chemists use the scientific method to perform experiments, pose hypotheses, and formulate laws and develop theories, so that they can better understand the behavior of the natural world. To do so, they operate in the macroscopic, microscopic, and symbolic domains. Chemists measure, analyze, purify, and synthesize a wide variety of substances that are important to our lives.

### 1.2 Phases and Classification of Matter

Matter is anything that occupies space and has mass. The basic building block of matter is the atom, the smallest unit of an element that can enter into combinations with atoms of the same or other elements. In many substances, atoms are combined into molecules. On earth, matter commonly exists in three states: solids, of fixed shape and volume; liquids, of variable shape but fixed volume; and gases, of variable shape and volume. Under high-temperature conditions, matter also can exist as a plasma. Most matter is a mixture: It is composed of two or more types of matter that can be present in varying amounts and can be separated by physical means. Heterogeneous mixtures vary in composition from point to point; homogeneous mixtures have the same composition from point to point. Pure substances consist of only one type of matter. A pure substance can be an element, which consists of only one type of atom and cannot be broken down by a chemical change, or a compound, which consists of two or more types of atoms.

### 1.3 Physical and Chemical Properties

All substances have distinct physical and chemical properties, and may undergo physical or chemical changes. Physical properties, such as hardness and boiling point, and physical changes, such as melting or freezing, do not involve a change in the composition of matter. Chemical properties, such flammability and acidity, and chemical changes, such as rusting, involve production of matter that differs from that present beforehand.

Measurable properties fall into one of two categories. Extensive properties depend on the amount of matter present, for example, the mass of gold. Intensive properties do not depend on the amount of matter present, for example, the density of gold. Heat is an example of an extensive property, and temperature is an example of an intensive property.

### 1.4 Measurements

Measurements provide quantitative information that is critical in studying and practicing chemistry. Each measurement has an amount, a unit for comparison, and an uncertainty. Measurements can be represented in either decimal or scientific notation. Scientists primarily use SI (International System) units such as meters, seconds, and kilograms, as well as derived units, such as liters (for volume) and $\mathrm{g} / \mathrm{cm}^{3}$ (for density). In many cases, it is convenient

to use prefixes that yield fractional and multiple units, such as microseconds ( $10^{-6}$ seconds) and megahertz ( $10^{6}$ hertz), respectively.

# 1.5 Measurement Uncertainty, Accuracy, and Precision 

Quantities can be defined or measured. Measured quantities have an associated uncertainty that is represented by the number of significant figures in the quantity's number. The uncertainty of a calculated quantity depends on the uncertainties in the quantities used in the calculation and is reflected in how the value is rounded. Quantities are characterized with regard to accuracy (closeness to a true or accepted value) and precision (variation among replicate measurement results).

### 1.6 Mathematical Treatment of Measurement Results

Measurements are made using a variety of units. It is often useful or necessary to convert a measured quantity from one unit into another. These conversions are accomplished using unit conversion factors, which are derived by simple applications of a mathematical approach called the factor-label method or dimensional analysis. This strategy is also employed to calculate sought quantities using measured quantities and appropriate mathematical relations.

## Exercises

### 1.1 Chemistry in Context

1. Explain how you could experimentally determine whether the outside temperature is higher or lower than $0^{\circ} \mathrm{C}$ $\left(32^{\circ} \mathrm{F}\right)$ without using a thermometer.
2. Identify each of the following statements as being most similar to a hypothesis, a law, or a theory. Explain your reasoning.
(a) Falling barometric pressure precedes the onset of bad weather.
(b) All life on earth has evolved from a common, primitive organism through the process of natural selection.
(c) My truck's gas mileage has dropped significantly, probably because it's due for a tune-up.
3. Identify each of the following statements as being most similar to a hypothesis, a law, or a theory. Explain your reasoning.
(a) The pressure of a sample of gas is directly proportional to the temperature of the gas.
(b) Matter consists of tiny particles that can combine in specific ratios to form substances with specific properties.
(c) At a higher temperature, solids (such as salt or sugar) will dissolve better in water.
4. Identify each of the underlined items as a part of either the macroscopic domain, the microscopic domain, or the symbolic domain of chemistry. For any in the symbolic domain, indicate whether they are symbols for a macroscopic or a microscopic feature.
(a) The mass of a lead pipe is 14 lb .
(b) The mass of a certain chlorine atom is 35 amu .
(c) A bottle with a label that reads $\underline{\mathrm{Al}}$ contains aluminum metal.
(d) $\underline{\mathrm{Al}}$ is the symbol for an aluminum atom.

5. Identify each of the underlined items as a part of either the macroscopic domain, the microscopic domain, or the symbolic domain of chemistry. For those in the symbolic domain, indicate whether they are symbols for a macroscopic or a microscopic feature.
(a) A certain molecule contains one $\underline{\mathrm{H}}$ atom and one Cl atom.
(b) Copper wire has a density of about $8 \mathrm{~g} / \mathrm{cm}^{3}$.
(c) The bottle contains 15 grams of Ni powder.
(d) A sulfur molecule is composed of eight sulfur atoms.
6. According to one theory, the pressure of a gas increases as its volume decreases because the molecules in the gas have to move a shorter distance to hit the walls of the container. Does this theory follow a macroscopic or microscopic description of chemical behavior? Explain your answer.
7. The amount of heat required to melt 2 lbs of ice is twice the amount of heat required to melt 1 lb of ice. Is this observation a macroscopic or microscopic description of chemical behavior? Explain your answer.

# 1.2 Phases and Classification of Matter 

8. Why is an object's mass, rather than its weight, used to indicate the amount of matter it contains?
9. What properties distinguish solids from liquids? Liquids from gases? Solids from gases?
10. How does a heterogeneous mixture differ from a homogeneous mixture? How are they similar?
11. How does a homogeneous mixture differ from a pure substance? How are they similar?
12. How does an element differ from a compound? How are they similar?
13. How do molecules of elements and molecules of compounds differ? In what ways are they similar?
14. How does an atom differ from a molecule? In what ways are they similar?
15. Many of the items you purchase are mixtures of pure compounds. Select three of these commercial products and prepare a list of the ingredients that are pure compounds.
16. Classify each of the following as an element, a compound, or a mixture:
(a) copper
(b) water
(c) nitrogen
(d) sulfur
(e) air
(f) sucrose
(g) a substance composed of molecules each of which contains two iodine atoms
(h) gasoline
17. Classify each of the following as an element, a compound, or a mixture:
(a) iron
(b) oxygen
(c) mercury oxide
(d) pancake syrup
(e) carbon dioxide
(f) a substance composed of molecules each of which contains one hydrogen atom and one chlorine atom
(g) baking soda
(h) baking powder
18. A sulfur atom and a sulfur molecule are not identical. What is the difference?

19. How are the molecules in oxygen gas, the molecules in hydrogen gas, and water molecules similar? How do they differ?
20. Why are astronauts in space said to be "weightless," but not "massless"?
21. Prepare a list of the principal chemicals consumed and produced during the operation of an automobile.
22. Matter is everywhere around us. Make a list by name of fifteen different kinds of matter that you encounter every day. Your list should include (and label at least one example of each) the following: a solid, a liquid, a gas, an element, a compound, a homogenous mixture, a heterogeneous mixture, and a pure substance.
23. When elemental iron corrodes it combines with oxygen in the air to ultimately form red brown iron(III) oxide called rust. (a) If a shiny iron nail with an initial mass of 23.2 g is weighed after being coated in a layer of rust, would you expect the mass to have increased, decreased, or remained the same? Explain. (b) If the mass of the iron nail increases to 24.1 g , what mass of oxygen combined with the iron?
24. As stated in the text, convincing examples that demonstrate the law of conservation of matter outside of the laboratory are few and far between. Indicate whether the mass would increase, decrease, or stay the same for the following scenarios where chemical reactions take place:
(a) Exactly one pound of bread dough is placed in a baking tin. The dough is cooked in an oven at $350^{\circ} \mathrm{F}$ releasing a wonderful aroma of freshly baked bread during the cooking process. Is the mass of the baked loaf less than, greater than, or the same as the one pound of original dough? Explain.
(b) When magnesium burns in air a white flaky ash of magnesium oxide is produced. Is the mass of magnesium oxide less than, greater than, or the same as the original piece of magnesium? Explain.
(c) Antoine Lavoisier, the French scientist credited with first stating the law of conservation of matter, heated a mixture of tin and air in a sealed flask to produce tin oxide. Did the mass of the sealed flask and contents decrease, increase, or remain the same after the heating?
25. Yeast converts glucose to ethanol and carbon dioxide during anaerobic fermentation as depicted in the simple chemical equation here:
glucose $\longrightarrow$ ethanol + carbon dioxide
(a) If 200.0 g of glucose is fully converted, what will be the total mass of ethanol and carbon dioxide produced?
(b) If the fermentation is carried out in an open container, would you expect the mass of the container and contents after fermentation to be less than, greater than, or the same as the mass of the container and contents before fermentation? Explain.
(c) If 97.7 g of carbon dioxide is produced, what mass of ethanol is produced?

# 1.3 Physical and Chemical Properties 

26. Classify the six underlined properties in the following paragraph as chemical or physical:

Fluorine is a pale yellow gas that reacts with most substances. The free element melts at $-220^{\circ} \mathrm{C}$ and boils at -188 ${ }^{\circ} \mathrm{C}$. Finely divided metals burn in fluorine with a bright flame. Nineteen grams of fluorine will react with 1.0 gram of hydrogen.
27. Classify each of the following changes as physical or chemical:
(a) condensation of steam
(b) burning of gasoline
(c) souring of milk
(d) dissolving of sugar in water
(e) melting of gold

28. Classify each of the following changes as physical or chemical:
(a) coal burning
(b) ice melting
(c) mixing chocolate syrup with milk
(d) explosion of a firecracker
(e) magnetizing of a screwdriver
29. The volume of a sample of oxygen gas changed from 10 mL to 11 mL as the temperature changed. Is this a chemical or physical change?
30. A 2.0 -liter volume of hydrogen gas combined with 1.0 liter of oxygen gas to produce 2.0 liters of water vapor. Does oxygen undergo a chemical or physical change?
31. Explain the difference between extensive properties and intensive properties.
32. Identify the following properties as either extensive or intensive.
(a) volume
(b) temperature
(c) humidity
(d) heat
(e) boiling point
33. The density (d) of a substance is an intensive property that is defined as the ratio of its mass (m) to its volume (V). density $=\frac{\text { mass }}{\text { volume }} \quad \mathrm{d}=\frac{\mathrm{m}}{\mathrm{V}}$

Considering that mass and volume are both extensive properties, explain why their ratio, density, is intensive.

# 1.4 Measurements 

34. Is one liter about an ounce, a pint, a quart, or a gallon?
35. Is a meter about an inch, a foot, a yard, or a mile?
36. Indicate the SI base units or derived units that are appropriate for the following measurements:
(a) the length of a marathon race ( 26 miles 385 yards)
(b) the mass of an automobile
(c) the volume of a swimming pool
(d) the speed of an airplane
(e) the density of gold
(f) the area of a football field
(g) the maximum temperature at the South Pole on April 1, 1913
37. Indicate the SI base units or derived units that are appropriate for the following measurements:
(a) the mass of the moon
(b) the distance from Dallas to Oklahoma City
(c) the speed of sound
(d) the density of air
(e) the temperature at which alcohol boils
(f) the area of the state of Delaware
(g) the volume of a flu shot or a measles vaccination

38. Give the name and symbol of the prefixes used with SI units to indicate multiplication by the following exact quantities.
(a) $10^{3}$
(b) $10^{-2}$
(c) 0.1
(d) $10^{-3}$
(e) $1,000,000$
(f) 0.000001
39. Give the name of the prefix and the quantity indicated by the following symbols that are used with SI base units.
(a) c
(b) d
(c) G
(d) k
(e) m
(f) n
(g) p
(h) T
40. A large piece of jewelry has a mass of 132.6 g . A graduated cylinder initially contains 48.6 mL water. When the jewelry is submerged in the graduated cylinder, the total volume increases to 61.2 mL .
(a) Determine the density of this piece of jewelry.
(b) Assuming that the jewelry is made from only one substance, what substance is it likely to be? Explain.
41. Visit this PhET density simulation (http://openstaxcollege.org/I/16phetmasvolden) and select the Same Volume Blocks.
(a) What are the mass, volume, and density of the yellow block?
(b) What are the mass, volume and density of the red block?
(c) List the block colors in order from smallest to largest mass.
(d) List the block colors in order from lowest to highest density.
(e) How are mass and density related for blocks of the same volume?
42. Visit this PhET density simulation (http://openstaxcollege.org/I/16phetmasvolden) and select Custom Blocks and then My Block.
(a) Enter mass and volume values for the block such that the mass in kg is less than the volume in L. What does the block do? Why? Is this always the case when mass < volume?
(b) Enter mass and volume values for the block such that the mass in kg is more than the volume in L. What does the block do? Why? Is this always the case when mass > volume?
(c) How would (a) and (b) be different if the liquid in the tank were ethanol instead of water?
(d) How would (a) and (b) be different if the liquid in the tank were mercury instead of water?

43. Visit this PhET density simulation (http://openstaxcollege.org/I/16phetmasvolden) and select Mystery Blocks.
(a) Pick one of the Mystery Blocks and determine its mass, volume, density, and its likely identity.
(b) Pick a different Mystery Block and determine its mass, volume, density, and its likely identity.
(c) Order the Mystery Blocks from least dense to most dense. Explain.

# 1.5 Measurement Uncertainty, Accuracy, and Precision 

44. Express each of the following numbers in scientific notation with correct significant figures:
(a) 711.0
(b) 0.239
(c) 90743
(d) 134.2
(e) 0.05499
(f) 10000.0
(g) 0.000000738592
45. Express each of the following numbers in exponential notation with correct significant figures:
(a) 704
(b) 0.03344
(c) 547.9
(d) 22086
(e) 1000.00
(f) 0.0000000651
(g) 0.007157
46. Indicate whether each of the following can be determined exactly or must be measured with some degree of uncertainty:
(a) the number of eggs in a basket
(b) the mass of a dozen eggs
(c) the number of gallons of gasoline necessary to fill an automobile gas tank
(d) the number of cm in 2 m
(e) the mass of a textbook
(f) the time required to drive from San Francisco to Kansas City at an average speed of $53 \mathrm{mi} / \mathrm{h}$
47. Indicate whether each of the following can be determined exactly or must be measured with some degree of uncertainty:
(a) the number of seconds in an hour
(b) the number of pages in this book
(c) the number of grams in your weight
(d) the number of grams in 3 kilograms
(e) the volume of water you drink in one day
(f) the distance from San Francisco to Kansas City

48. How many significant figures are contained in each of the following measurements?
(a) 38.7 g
(b) $2 \times 10^{18} \mathrm{~m}$
(c) $3,486,002 \mathrm{~kg}$
(d) $9.74150 \times 10^{-4} \mathrm{~J}$
(e) $0.0613 \mathrm{~cm}^{3}$
(f) 17.0 kg
(g) $0.01400 \mathrm{~g} / \mathrm{mL}$
49. How many significant figures are contained in each of the following measurements?
(a) 53 cm
(b) $2.05 \times 10^{8} \mathrm{~m}$
(c) $86,002 \mathrm{~J}$
(d) $9.740 \times 10^{4} \mathrm{~m} / \mathrm{s}$
(e) $10.0613 \mathrm{~m}^{3}$
(f) $0.17 \mathrm{~g} / \mathrm{mL}$
(g) 0.88400 s
50. The following quantities were reported on the labels of commercial products. Determine the number of significant figures in each.
(a) 0.0055 g active ingredients
(b) 12 tablets
(c) $3 \%$ hydrogen peroxide
(d) 5.5 ounces
(e) 473 mL
(f) $1.75 \%$ bismuth
(g) $0.001 \%$ phosphoric acid
(h) $99.80 \%$ inert ingredients
51. Round off each of the following numbers to two significant figures:
(a) 0.436
(b) 9.000
(c) 27.2
(d) 135
(e) $1.497 \times 10^{-3}$
(f) 0.445

52. Round off each of the following numbers to two significant figures:
(a) 517
(b) 86.3
(c) $6.382 \times 10^{3}$
(d) 5.0008
(e) 22.497
(f) 0.885
53. Perform the following calculations and report each answer with the correct number of significant figures.
(a) $628 \times 342$
(b) $\left(5.63 \times 10^{2}\right) \times\left(7.4 \times 10^{3}\right)$
(c) $\frac{28.0}{13.483}$
(d) $8119 \times 0.000023$
(e) $14.98+27,340+84.7593$
(f) $42.7+0.259$
54. Perform the following calculations and report each answer with the correct number of significant figures.
(a) $62.8 \times 34$
(b) $0.147+0.0066+0.012$
(c) $38 \times 95 \times 1.792$
(d) $15-0.15-0.6155$
(e) $8.78 \times\left(\frac{0.0500}{0.478}\right)$
(f) $140+7.68+0.014$
(g) $28.7-0.0483$
(h) $\frac{(88.5-87.57)}{45.13}$

55. Consider the results of the archery contest shown in this figure.
(a) Which archer is most precise?
(b) Which archer is most accurate?
(c) Who is both least precise and least accurate?
![Image](Chapter_1_images/img-37.jpeg)
56. Classify the following sets of measurements as accurate, precise, both, or neither.
(a) Checking for consistency in the weight of chocolate chip cookies: $17.27 \mathrm{~g}, 13.05 \mathrm{~g}, 19.46 \mathrm{~g}, 16.92 \mathrm{~g}$
(b) Testing the volume of a batch of $25-\mathrm{mL}$ pipettes: $27.02 \mathrm{~mL}, 26.99 \mathrm{~mL}, 26.97 \mathrm{~mL}, 27.01 \mathrm{~mL}$
(c) Determining the purity of gold: $99.9999 \%, 99.9998 \%, 99.9998 \%, 99.9999 \%$

# 1.6 Mathematical Treatment of Measurement Results 

57. Write conversion factors (as ratios) for the number of:
(a) yards in 1 meter
(b) liters in 1 liquid quart
(c) pounds in 1 kilogram
58. Write conversion factors (as ratios) for the number of:
(a) kilometers in 1 mile
(b) liters in 1 cubic foot
(c) grams in 1 ounce
59. The label on a soft drink bottle gives the volume in two units: 2.0 L and 67.6 fl oz. Use this information to derive a conversion factor between the English and metric units. How many significant figures can you justify in your conversion factor?
60. The label on a box of cereal gives the mass of cereal in two units: 978 grams and 34.5 oz. Use this information to find a conversion factor between the English and metric units. How many significant figures can you justify in your conversion factor?
61. Soccer is played with a round ball having a circumference between 27 and 28 in. and a weight between 14 and 16 oz. What are these specifications in units of centimeters and grams?
62. A woman's basketball has a circumference between 28.5 and 29.0 inches and a maximum weight of 20 ounces (two significant figures). What are these specifications in units of centimeters and grams?
63. How many milliliters of a soft drink are contained in a 12.0 -oz can?
64. A barrel of oil is exactly 42 gal . How many liters of oil are in a barrel?
65. The diameter of a red blood cell is about $3 \times 10^{-4} \mathrm{in}$. What is its diameter in centimeters?

66. The distance between the centers of the two oxygen atoms in an oxygen molecule is $1.21 \times 10^{-8} \mathrm{~cm}$. What is this distance in inches?
67. Is a 197-lb weight lifter light enough to compete in a class limited to those weighing 90 kg or less?
68. A very good 197-lb weight lifter lifted 192 kg in a move called the clean and jerk. What was the mass of the weight lifted in pounds?
69. Many medical laboratory tests are run using $5.0 \mu \mathrm{~L}$ blood serum. What is this volume in milliliters?
70. If an aspirin tablet contains 325 mg aspirin, how many grams of aspirin does it contain?
71. Use scientific (exponential) notation to express the following quantities in terms of the SI base units in Table 1.3:
(a) 0.13 g
(b) 232 Gg
(c) 5.23 pm
(d) 86.3 mg
(e) 37.6 cm
(f) $54 \mu \mathrm{~m}$
(g) 1 Ts
(h) 27 ps
(i) 0.15 mK
72. Complete the following conversions between SI units.
(a) $612 \mathrm{~g}=$ $\qquad$ mg
(b) $8.160 \mathrm{~m}=$ $\qquad$ cm
(c) $3779 \mu \mathrm{~g}=$ $\qquad$ g
(d) $781 \mathrm{~mL}=$ $\qquad$ L
(e) $4.18 \mathrm{~kg}=$ $\qquad$ g
(f) $27.8 \mathrm{~m}=$ $\qquad$ km
(g) $0.13 \mathrm{~mL}=$ $\qquad$ L
(h) $1738 \mathrm{~km}=$ $\qquad$ m
(i) $1.9 \mathrm{Gg}=$ $\qquad$ g
73. Gasoline is sold by the liter in many countries. How many liters are required to fill a 12.0-gal gas tank?
74. Milk is sold by the liter in many countries. What is the volume of exactly $1 / 2$ gal of milk in liters?
75. A long ton is defined as exactly 2240 lb . What is this mass in kilograms?
76. Make the conversion indicated in each of the following:
(a) the men's world record long jump, $29 \mathrm{ft} 4 \frac{1}{4}$ in., to meters
(b) the greatest depth of the ocean, about 6.5 mi , to kilometers
(c) the area of the state of Oregon, $96,981 \mathrm{mi}^{2}$, to square kilometers
(d) the volume of 1 gill (exactly 4 oz ) to milliliters
(e) the estimated volume of the oceans, $330,000,000 \mathrm{mi}^{3}$, to cubic kilometers.
(f) the mass of a 3525-lb car to kilograms
(g) the mass of a 2.3-oz egg to grams

77. Make the conversion indicated in each of the following:
(a) the length of a soccer field, 120 m (three significant figures), to feet
(b) the height of Mt. Kilimanjaro, at 19,565 ft, the highest mountain in Africa, to kilometers
(c) the area of an $8.5-\times 11$-inch sheet of paper in $\mathrm{cm}^{2}$
(d) the displacement volume of an automobile engine, $161 \mathrm{in} .^{3}$, to liters
(e) the estimated mass of the atmosphere, $5.6 \times 10^{15}$ tons, to kilograms
(f) the mass of a bushel of rye, 32.0 lb , to kilograms
(g) the mass of a 5.00-grain aspirin tablet to milligrams ( 1 grain $=0.00229$ oz)
78. Many chemistry conferences have held a 50-Trillion Angstrom Run (two significant figures). How long is this run in kilometers and in miles? $\left(1 \AA=1 \times 10^{-10} \mathrm{~m}\right)$
79. A chemist's 50-Trillion Angstrom Run (see Exercise 1.78) would be an archeologist's 10,900 cubit run. How long is one cubit in meters and in feet? $\left(1 \AA=1 \times 10^{-8} \mathrm{~cm}\right)$
80. The gas tank of a certain luxury automobile holds 22.3 gallons according to the owner's manual. If the density of gasoline is $0.8206 \mathrm{~g} / \mathrm{mL}$, determine the mass in kilograms and pounds of the fuel in a full tank.
81. As an instructor is preparing for an experiment, he requires 225 g phosphoric acid. The only container readily available is a $150-\mathrm{mL}$ Erlenmeyer flask. Is it large enough to contain the acid, whose density is $1.83 \mathrm{~g} / \mathrm{mL}$ ?
82. To prepare for a laboratory period, a student lab assistant needs 125 g of a compound. A bottle containing 1/4 lb is available. Did the student have enough of the compound?
83. A chemistry student is 159 cm tall and weighs 45.8 kg . What is her height in inches and weight in pounds?
84. In a recent Grand Prix, the winner completed the race with an average speed of $229.8 \mathrm{~km} / \mathrm{h}$. What was his speed in miles per hour, meters per second, and feet per second?
85. Solve these problems about lumber dimensions.
(a) To describe to a European how houses are constructed in the US, the dimensions of "two-by-four" lumber must be converted into metric units. The thickness $\times$ width $\times$ length dimensions are $1.50 \mathrm{in} . \times 3.50 \mathrm{in} . \times 8.00 \mathrm{ft}$ in the US. What are the dimensions in $\mathrm{cm} \times \mathrm{cm} \times \mathrm{m}$ ?
(b) This lumber can be used as vertical studs, which are typically placed 16.0 in. apart. What is that distance in centimeters?
86. The mercury content of a stream was believed to be above the minimum considered safe-1 part per billion (ppb) by weight. An analysis indicated that the concentration was 0.68 parts per billion. What quantity of mercury in grams was present in 15.0 L of the water, the density of which is $0.998 \mathrm{~g} / \mathrm{ml}$ ? ( $1 \mathrm{ppb} \mathrm{Hg}=\frac{1 \mathrm{ng} \mathrm{Hg}}{1 \mathrm{~g} \text { water }}$ )
87. Calculate the density of aluminum if $27.6 \mathrm{~cm}^{3}$ has a mass of 74.6 g .
88. Osmium is one of the densest elements known. What is its density if 2.72 g has a volume of $0.121 \mathrm{~cm}^{3}$ ?
89. Calculate these masses.
(a) What is the mass of $6.00 \mathrm{~cm}^{3}$ of mercury, density $=13.5939 \mathrm{~g} / \mathrm{cm}^{3}$ ?
(b) What is the mass of 25.0 mL octane, density $=0.702 \mathrm{~g} / \mathrm{cm}^{3}$ ?
90. Calculate these masses.
(a) What is the mass of $4.00 \mathrm{~cm}^{3}$ of sodium, density $=0.97 \mathrm{~g} / \mathrm{cm}^{3}$ ?
(b) What is the mass of 125 mL gaseous chlorine, density $=3.16 \mathrm{~g} / \mathrm{L}$ ?
91. Calculate these volumes.
(a) What is the volume of 25 g iodine, density $=4.93 \mathrm{~g} / \mathrm{cm}^{3}$ ?
(b) What is the volume of 3.28 g gaseous hydrogen, density $=0.089 \mathrm{~g} / \mathrm{L}$ ?

92. Calculate these volumes.
(a) What is the volume of 11.3 g graphite, density $=2.25 \mathrm{~g} / \mathrm{cm}^{3}$ ?
(b) What is the volume of 39.657 g bromine, density $=2.928 \mathrm{~g} / \mathrm{cm}^{3}$ ?
93. Convert the boiling temperature of gold, $2966^{\circ} \mathrm{C}$, into degrees Fahrenheit and kelvin.
94. Convert the temperature of scalding water, $54^{\circ} \mathrm{C}$, into degrees Fahrenheit and kelvin.
95. Convert the temperature of the coldest area in a freezer, $-10^{\circ} \mathrm{F}$, to degrees Celsius and kelvin.
96. Convert the temperature of dry ice, $-77^{\circ} \mathrm{C}$, into degrees Fahrenheit and kelvin.
97. Convert the boiling temperature of liquid ammonia, $-28.1^{\circ} \mathrm{F}$, into degrees Celsius and kelvin.
98. The label on a pressurized can of spray disinfectant warns against heating the can above $130^{\circ} \mathrm{F}$. What are the corresponding temperatures on the Celsius and kelvin temperature scales?
99. The weather in Europe was unusually warm during the summer of 1995. The TV news reported temperatures as high as $45^{\circ} \mathrm{C}$. What was the temperature on the Fahrenheit scale?

